In [11]:
# import necessary packages

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
import warnings
from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, ParameterSampler, ParameterGrid, StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, log_loss, brier_score_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
from scipy.special import expit
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback
from scikeras.wrappers import KerasClassifier
import xgboost as xgb

display(HTML("""
<style>
.output {
    max-height: 300px;
    overflow-y: scroll;
}
</style>
"""))


In [8]:
# set global variables

rootDirectory = '/Users/dthomas/Desktop/Coding/NCAAB/Latest_Commit'
startingSeason = 2003  # minimum is 2003 with current dataset
currentSeason = 2025  # based on when the season concludes


In [4]:
# calculate end-of-year totals for first season of data

# read in raw game data
rawGamesDF = pd.read_csv(rootDirectory + '/Data/Regular_Season_Detailed_Results.csv')

# convert from legacy Massey IDs to Massey URLs
massey_conversion_df = pd.read_csv(rootDirectory + '/Data/Team_Name_Conversion.csv')
massey_id_to_massey_url_dict = dict(zip(massey_conversion_df.dropna(subset=['Massey ID (Legacy)'])['Massey ID (Legacy)'], massey_conversion_df.dropna(subset=['Massey ID (Legacy)'])['Massey URL']))
rawGamesDF['WTeamID'] = rawGamesDF['WTeamID'].map(massey_id_to_massey_url_dict)
rawGamesDF['LTeamID'] = rawGamesDF['LTeamID'].map(massey_id_to_massey_url_dict)

# identify close games
rawGamesDF['Close'] = np.logical_or(rawGamesDF.loc[:, 'NumOT'], rawGamesDF.loc[:, 'WScore'] - rawGamesDF.loc[:, 'LScore'] <= 3)

# to prevent division by zero, set a small positive minimum for field goal attempts, 3-point field goal attempts, free-throw attempts, and turnovers
columns_to_convert = ['WFGA', 'LFGA', 'WFGA3', 'LFGA3', 'WFTA', 'LFTA', 'WTO', 'LTO']
rawGamesDF[columns_to_convert] = rawGamesDF[columns_to_convert].astype(float)
rawGamesDF.loc[:, 'WFGA'] = rawGamesDF.loc[:, 'WFGA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFGA'] = rawGamesDF.loc[:, 'LFGA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WFGA3'] = rawGamesDF.loc[:, 'WFGA3'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFGA3'] = rawGamesDF.loc[:, 'LFGA3'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WFTA'] = rawGamesDF.loc[:, 'WFTA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFTA'] = rawGamesDF.loc[:, 'LFTA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WTO'] = rawGamesDF.loc[:, 'WTO'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LTO'] = rawGamesDF.loc[:, 'LTO'].map(lambda x: max(x, 2 ** -64))

# add stop-sign row at the end of rawGamesDF
rawGamesDF.loc[len(rawGamesDF), 'Season'] = 9999

# read in set of all teams
teams = pd.read_csv(rootDirectory + '/Data/Team_Name_Conversion.csv').iloc[:, 0:2]

# initialize weights for strength of schedule calculations (increase from 0 to an upper limit of 1)
GamesPlayedWeights = .00390625 * (((np.array(list(range(45)))) * (np.array(list(range(45)))) * (np.array(list(range(45)))) * (np.array(list(range(45))))) / (((.0625 * (np.array(list(range(45)))) * (np.array(list(range(45))))) + 1) * ((.0625 * (np.array(list(range(45)))) * (np.array(list(range(45))))) + 1)))  # used to weight the importance of opponents' records by the number of games they've played
# GamesPlayedWeights = 0.00390625 * ((np.array(list(range(45))) / 1.0) ** 4) / ((((0.0625 * (np.array(list(range(45))) / 1.0) ** 2)) + 1) ** 2)

# initialize dictionary to store refined data for each season
seasonTotalsDict = {}

# disable performance warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# set starting season and day before first season and day
season = 1891  # set to arbitrary year prior to start of data
upToMinuteSeasonTotals = pd.DataFrame({})
dayNum = -1

# iterate through each game in rawGamesDF, starting from startingSeason
for index in range((rawGamesDF['Season'] == startingSeason).idxmax(), len(rawGamesDF)):

    # if this is the first game of a new season
    if rawGamesDF.iloc[index, 0] != season:

        # if at least 1 season's worth of games have been processed
        if season > 1891:
            # make end-of-season calculations
            upToMinuteSeasonTotals.loc[:, 'WinPct'] = upToMinuteSeasonTotals.loc[:, 'Wins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FlipCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'FlipCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'WinCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'WinCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'LoseCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'LoseCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsPerGame'] = upToMinuteSeasonTotals.loc[:, 'Points'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'PointsAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'PointsPerGame'] - upToMinuteSeasonTotals.loc[:, 'PointsAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGMPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGMAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGMAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGMMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGMPerGame'] - upToMinuteSeasonTotals.loc[:, 'FGMAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGAPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGAAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGAAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGAMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGAPerGame'] - upToMinuteSeasonTotals.loc[:, 'FGAAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGPctMargin'] = upToMinuteSeasonTotals.loc[:, 'FGPct'] - upToMinuteSeasonTotals.loc[:, 'FGPctAllowed']
            upToMinuteSeasonTotals.loc[:, 'FGM3PerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGM3AllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3Allowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGM3MarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3PerGame'] - upToMinuteSeasonTotals.loc[:, 'FGM3AllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGA3PerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGA3AllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3Allowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGA3MarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3PerGame'] - upToMinuteSeasonTotals.loc[:, 'FGA3AllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FG3PctMargin'] = upToMinuteSeasonTotals.loc[:, 'FGPct'] - upToMinuteSeasonTotals.loc[:, 'FGPctAllowed']
            upToMinuteSeasonTotals.loc[:, 'FTMPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTM'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTMAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTMAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTMMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTMPerGame'] - upToMinuteSeasonTotals.loc[:, 'FTMAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FTAPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTA'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTAAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTAAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTAMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTAPerGame'] - upToMinuteSeasonTotals.loc[:, 'FTAAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FTPctMargin'] = upToMinuteSeasonTotals.loc[:, 'FTPct'] - upToMinuteSeasonTotals.loc[:, 'FTPctAllowed']
            upToMinuteSeasonTotals.loc[:, 'RebPerGame'] = (upToMinuteSeasonTotals.loc[:, 'OReb'] + upToMinuteSeasonTotals.loc[:, 'DReb']) / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'RebAllowedPerGame'] = (upToMinuteSeasonTotals.loc[:, 'ORebAllowed'] + upToMinuteSeasonTotals.loc[:, 'DRebAllowed']) / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'RebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'RebPerGame'] - upToMinuteSeasonTotals.loc[:, 'RebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'ORebPerGame'] = upToMinuteSeasonTotals.loc[:, 'OReb'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'ORebAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'ORebAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'ORebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'ORebPerGame'] - upToMinuteSeasonTotals.loc[:, 'ORebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'DRebPerGame'] = upToMinuteSeasonTotals.loc[:, 'DReb'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'DRebAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'DRebAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'DRebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'DRebPerGame'] - upToMinuteSeasonTotals.loc[:, 'DRebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'AstPerGame'] = upToMinuteSeasonTotals.loc[:, 'Ast'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'AstAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'AstAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'AstMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'AstPerGame'] - upToMinuteSeasonTotals.loc[:, 'AstAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'TOPerGame'] = upToMinuteSeasonTotals.loc[:, 'TO'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'TOAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'TOAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'TOMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'TOPerGame'] - upToMinuteSeasonTotals.loc[:, 'TOAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatioMargin'] = upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio'] - upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatioAllowed']
            upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)Margin'] = upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)'] - upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)Allowed']
            upToMinuteSeasonTotals.loc[:, 'StlPerGame'] = upToMinuteSeasonTotals.loc[:, 'Stl'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'StlAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'StlAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'StlMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'StlPerGame'] - upToMinuteSeasonTotals.loc[:, 'StlAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'BlkPerGame'] = upToMinuteSeasonTotals.loc[:, 'Blk'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'BlkAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'BlkAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'BlkMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'BlkPerGame'] - upToMinuteSeasonTotals.loc[:, 'BlkAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'PFPerGame'] = upToMinuteSeasonTotals.loc[:, 'PF'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PFAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'PFAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PFMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'PFPerGame'] - upToMinuteSeasonTotals.loc[:, 'PFAllowedPerGame']
            # strength of schedule
            for teamID in upToMinuteSeasonTotals.index:
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'WinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFlipCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FlipCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppWinCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'WinCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppLoseCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'LoseCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3PerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3PerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3AllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3AllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3MarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3MarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3PerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3PerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3AllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3AllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3MarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3MarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3Pct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3PctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3PctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatioMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)Now'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)AllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)MarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Margin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFMarginPerGame'].mean()

            # memoize end-of-season totals
            seasonTotalsDict[season] = upToMinuteSeasonTotals.copy()
            # write previous season totals
            seasonTotalsDict[int(season)].to_csv(rootDirectory + '/Data/Team_Stats_Season_Totals' + '/Team_Stats_Season_Totals_' + str(int(season)) + '.csv')
            print('Team_Stats_Season_Totals_' + str(int(season)) + '.csv written')
        # advance to next season
        season = rawGamesDF.iloc[index, 0]
        if season == startingSeason + 1:
            break
        print('Season:', int(season), '   Current time:', str(datetime.now())[:-7])
        # reset day number
        dayNum = -1
        # reset season totals
        upToMinuteSeasonTotals = pd.DataFrame(columns=['Team ID',  # 4-digit numerical team identification number
                                                       # season totals
                                                       'GamesPlayed',  # total games played during the current season
                                                       'Wins',  # total wins during the current season
                                                       'CloseGames',  # total close games played during the current season
                                                       'FlipCloseWins',  # total wins during the current season (had the results of all of their close games been flipped)
                                                       'WinCloseWins',  # total wins during the current season (had they won all of their close games)
                                                       'LoseCloseWins',  # total wins during the current season (had they lost all of their close games)
                                                       'Points',  # total points scored during the current season
                                                       'PointsAllowed',  # total points allowed during the current season
                                                       'FGM',  # total field goals made during the current season
                                                       'FGMAllowed',  # total field goals allowed during the current season
                                                       'FGA',  # total field goal attempts during the current season
                                                       'FGAAllowed',  # total field goal attempts allowed during the current season
                                                       'FGPct',  # total field goals made divided by total field goal attempts during the current season
                                                       'FGPctAllowed',  # total field goals allowed divided by total field goal attempts allowed during the current season
                                                       'FGM3',  # total 3-point field goals made during the current season
                                                       'FGM3Allowed',  # total 3-point field goals allowed during the current season
                                                       'FGA3',  # total 3 point field goal attempts during the current season
                                                       'FGA3Allowed',  # total 3-point field goal attempts allowed during the current season
                                                       'FG3Pct',  # total 3-point field goals made divided by total 3-point field goal attempts during the current season
                                                       'FG3PctAllowed',  # total 3-point field goals allowed divided by total 3-point field goal attempts allowed during the current season
                                                       'FTM',  # total free-throws made during the current season
                                                       'FTMAllowed',  # total successful free-throws allowed during the current season
                                                       'FTA',  # total free-throw attempts during the current season
                                                       'FTAAllowed',  # total free-throw attempts allowed during the current season
                                                       'FTPct',  # total free-throws made divided by total free-throw attempts during the current season
                                                       'FTPctAllowed',  # total successful free-throws allowed divided by total free-throw attempts allowed during the current season
                                                       'OReb',  # total offensive rebounds during the current season
                                                       'ORebAllowed',  # total offensive rebounds allowed during the current season
                                                       'DReb',  # total defensive rebounds during the current season
                                                       'DRebAllowed',  # total defensive rebounds allowed during the current season
                                                       'Ast',  # total assists during the current season
                                                       'AstAllowed',  # total assists allowed during the current season
                                                       'TO',  # total turnovers during the current season
                                                       'TOAllowed',  # total turnovers forced during the current season
                                                       'AvgAst/TORatio',  # total assists divided by total turnovers during the current season
                                                       'AvgAst/TORatioAllowed',  # total assists allowed divided by total turnovers forced during the current season
                                                       'AvgAst/TORatio(alt)',  # average of assist/turnover ratios during the current season (calculated by averaging each game's assist/turnover ratio, not by dividing total assists / total turnovers)
                                                       'AvgAst/TORatio(alt)Allowed',  # average of opposing assist/turnover ratios during the current season (calculated by averaging each game's assist/turnover ratio, not by dividing total assists / total turnovers)
                                                       'Stl',  # total steals during the current season
                                                       'StlAllowed',  # total steals allowed during the current season
                                                       'Blk',  # total blocks during the current season
                                                       'BlkAllowed',  # total blocks allowed during the current season
                                                       'PF',  # total fouls committed during the current season
                                                       'PFAllowed',  # total fouls absorbed during the current season
                                                       # strength of schedule
                                                       'ListOpponentsTeamID',  # list of opponents' 4-digit team identification numbers
                                                       'SoSSumOppGamesPlayedWeightsDenominator',  # sum of weights applied to opponents' stats based on number of games they'd played prior to their contest against this team
                                                       'SoSWeightedAvgOpponentsWinPct',  # weighted average of opponents' win percentages entering contest against this team
                                                       'SoSWeightedAvgOpponentsFlipCloseWinPct',  # weighted average of opponents' win percentages (had the results of all of their close games been flipped) entering contest against this team
                                                       'SoSWeightedAvgOpponentsWinCloseWinPct',  # weighted average of opponents' win percentages (had they won all of their close games) entering contest against this team
                                                       'SoSWeightedAvgOpponentsLoseCloseWinPct',  # weighted average of opponents' win percentages (had they lost all of their close games) entering contest against this team
                                                       'SoSWeightedAvgOpponentsPointsPerGame',  # weighted average of opponents' average points per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPointsAllowedPerGame',  # weighted average of opponents' average points per game allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGMPerGame',  # weighted average of opponents' average field goals made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGMAllowedPerGame',  # weighted average of opponents' average field goals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGAPerGame',  # weighted average of opponents' average field goal attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGAAllowedPerGame',  # weighted average of opponents' average field goal attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGPct',  # weighted average of opponents' season-long field goal percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGPctAllowed',  # weighted average of opponents' season-long field goal percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGM3PerGame',  # weighted average of opponents' average 3-point field goals made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGM3AllowedPerGame',  # weighted average of opponents' average 3-point field goals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGA3PerGame',  # weighted average of opponents' average 3-point field goal attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGA3AllowedPerGame',  # weighted average of opponents' average 3-point field goal attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFG3Pct',  # weighted average of opponents' season-long 3-point field goal percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFG3PctAllowed',  # weighted average of opponents' season-long 3-point field goal percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTMPerGame',  # weighted average of opponents' average free throws made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTMAllowedPerGame',  # weighted average of opponents' average successful free throws allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTAPerGame',  # weighted average of opponents' average free throw attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTAAllowedPerGame',  # weighted average of opponents' average free throw attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTPct',  # weighted average of opponents' season-long free-throw percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTPctAllowed',  # weighted average of opponents' season-long free-throw percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsORebPerGame',  # weighted average of opponents' average offensive rebounds per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsORebAllowedPerGame',  # weighted average of opponents' average offensive rebounds allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsDRebPerGame',  # weighted average of opponents' average defensive rebounds per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsDRebAllowedPerGame',  # weighted average of opponents' average defensive rebounds allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAstPerGame',  # weighted average of opponents' average assists per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAstAllowedPerGame',  # weighted average of opponents' average assists allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsTOPerGame',  # weighted average of opponents' average turnovers per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsTOAllowedPerGame',  # weighted average of opponents' average turnovers forced per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAst/TORatio',  # weighted average of opponents' season-long assist/turnover ratios (total assists / total turnovers) entering contest against this team
                                                       'SoSWeightedAvgOpponentsAst/TORatioAllowed',  # weighted average of opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) entering contest against this team (calculated by averaging each team's total assists allowed divided by that team's total turnovers forced)
                                                       'SoSWeightedAvgOpponentsAst/TORatio(alt)',  # weighted average of opponents' average of assist/turnover ratios entering contest against this team (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
                                                       'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed',  # weighted average of opponents' average of assist/turnover ratios allowed entering contest against this team (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
                                                       'SoSWeightedAvgOpponentsStlPerGame',  # weighted average of opponents' average steals per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsStlAllowedPerGame',  # weighted average of opponents' average steals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsBlkPerGame',  # weighted average of opponents' average blocks per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsBlkAllowedPerGame',  # weighted average of opponents' average blocks allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPFPerGame',  # weighted average of opponents' average fouls committed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPFAllowedPerGame'])  # weighted average of opponents' average fouls absorbed per game entering contest against this team
        upToMinuteSeasonTotals.set_index('Team ID', inplace=True)
        for team in teams.iloc[:, 0]:
            upToMinuteSeasonTotals.loc[team] = {
                'GamesPlayed': 0, 'Wins': 0.0, 'CloseGames': 0.0, 'FlipCloseWins': 0.0, 
                'WinCloseWins': 0.0, 'LoseCloseWins': 0.0, 'Points': 0.0, 'PointsAllowed': 0.0, 
                'FGM': 0.0, 'FGMAllowed': 0.0, 'FGA': 0.0, 'FGAAllowed': 0.0, 'FGPct': 0.0, 
                'FGPctAllowed': 0.0, 'FGM3': 0.0, 'FGM3Allowed': 0.0, 'FGA3': 0.0, 
                'FGA3Allowed': 0.0, 'FG3Pct': 0.0, 'FG3PctAllowed': 0.0, 'FTM': 0.0, 
                'FTMAllowed': 0.0, 'FTA': 0.0, 'FTAAllowed': 0.0, 'FTPct': 0.0, 
                'FTPctAllowed': 0.0, 'OReb': 0.0, 'ORebAllowed': 0.0, 'DReb': 0.0, 
                'DRebAllowed': 0.0, 'Ast': 0.0, 'AstAllowed': 0.0, 'TO': 0.0, 'TOAllowed': 0.0, 
                'AvgAst/TORatio': 0.0, 'AvgAst/TORatioAllowed': 0.0, 'AvgAst/TORatio(alt)': 0.0, 
                'AvgAst/TORatio(alt)Allowed': 0.0, 'Stl': 0.0, 'StlAllowed': 0.0, 'Blk': 0.0, 
                'BlkAllowed': 0.0, 'PF': 0.0, 'PFAllowed': 0.0, 'ListOpponentsTeamID': [], 
                'SoSSumOppGamesPlayedWeightsDenominator': 0.0, 'SoSWeightedAvgOpponentsWinPct': 0.0, 
                'SoSWeightedAvgOpponentsFlipCloseWinPct': 0.0, 'SoSWeightedAvgOpponentsWinCloseWinPct': 0.0, 
                'SoSWeightedAvgOpponentsLoseCloseWinPct': 0.0, 'SoSWeightedAvgOpponentsPointsPerGame': 0.0, 
                'SoSWeightedAvgOpponentsPointsAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGMPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGMAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGAPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGAAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGPct': 0.0, 
                'SoSWeightedAvgOpponentsFGPctAllowed': 0.0, 'SoSWeightedAvgOpponentsFGM3PerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGM3AllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGA3PerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGA3AllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFG3Pct': 0.0, 
                'SoSWeightedAvgOpponentsFG3PctAllowed': 0.0, 'SoSWeightedAvgOpponentsFTMPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFTMAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFTAPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFTAAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFTPct': 0.0, 
                'SoSWeightedAvgOpponentsFTPctAllowed': 0.0, 'SoSWeightedAvgOpponentsORebPerGame': 0.0, 
                'SoSWeightedAvgOpponentsORebAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsDRebPerGame': 0.0, 
                'SoSWeightedAvgOpponentsDRebAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsAstPerGame': 0.0, 
                'SoSWeightedAvgOpponentsAstAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsTOPerGame': 0.0, 
                'SoSWeightedAvgOpponentsTOAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsAst/TORatio': 0.0, 
                'SoSWeightedAvgOpponentsAst/TORatioAllowed': 0.0, 'SoSWeightedAvgOpponentsAst/TORatio(alt)': 0.0, 
                'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed': 0.0, 'SoSWeightedAvgOpponentsStlPerGame': 0.0, 
                'SoSWeightedAvgOpponentsStlAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsBlkPerGame': 0.0, 
                'SoSWeightedAvgOpponentsBlkAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsPFPerGame': 0.0, 
                'SoSWeightedAvgOpponentsPFAllowedPerGame': 0.0
            }

    # if this is the first game of a new day
    if rawGamesDF.iloc[index, 1] > dayNum:
        dayNum = rawGamesDF.iloc[index, 1]
        print('Season:', int(season), '   Day Number:', int(dayNum), '   Current Time:', str(datetime.now())[:-7])
        # update season totals
        startOfDaySeasonTotals = upToMinuteSeasonTotals.copy()

    # update winning team's up-to-the-minute season totals with results of game
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Wins'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'CloseGames'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FlipCloseWins'] += 1 - rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'WinCloseWins'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'LoseCloseWins'] += 1 - rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Points'] += rawGamesDF.loc[index, 'WScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PointsAllowed'] += rawGamesDF.loc[index, 'LScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] += rawGamesDF.loc[index, 'WFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] += rawGamesDF.loc[index, 'LFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] += rawGamesDF.loc[index, 'WFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] += rawGamesDF.loc[index, 'LFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] + rawGamesDF.loc[index, 'WFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] + rawGamesDF.loc[index, 'WFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] + rawGamesDF.loc[index, 'LFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] + rawGamesDF.loc[index, 'LFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] += rawGamesDF.loc[index, 'WFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] += rawGamesDF.loc[index, 'LFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] += rawGamesDF.loc[index, 'WFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] += rawGamesDF.loc[index, 'LFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] + rawGamesDF.loc[index, 'WFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] + rawGamesDF.loc[index, 'WFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] + rawGamesDF.loc[index, 'LFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] + rawGamesDF.loc[index, 'LFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] += rawGamesDF.loc[index, 'WFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] += rawGamesDF.loc[index, 'LFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] += rawGamesDF.loc[index, 'WFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] += rawGamesDF.loc[index, 'LFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] + rawGamesDF.loc[index, 'WFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] + rawGamesDF.loc[index, 'WFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] + rawGamesDF.loc[index, 'LFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] + rawGamesDF.loc[index, 'LFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'OReb'] += rawGamesDF.loc[index, 'WOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ORebAllowed'] += rawGamesDF.loc[index, 'LOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DReb'] += rawGamesDF.loc[index, 'WDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DRebAllowed'] += rawGamesDF.loc[index, 'LDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] += rawGamesDF.loc[index, 'WAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] += rawGamesDF.loc[index, 'LAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] += rawGamesDF.loc[index, 'WTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] += rawGamesDF.loc[index, 'LTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] + rawGamesDF.loc[index, 'WAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] + rawGamesDF.loc[index, 'WTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] + rawGamesDF.loc[index, 'LAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] + rawGamesDF.loc[index, 'LTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'WAst'] / rawGamesDF.loc[index, 'WTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'LAst'] / rawGamesDF.loc[index, 'LTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Stl'] += rawGamesDF.loc[index, 'WStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'StlAllowed'] += rawGamesDF.loc[index, 'LStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Blk'] += rawGamesDF.loc[index, 'WBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'BlkAllowed'] += rawGamesDF.loc[index, 'LBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PF'] += rawGamesDF.loc[index, 'WPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PFAllowed'] += rawGamesDF.loc[index, 'LPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ListOpponentsTeamID'].append(rawGamesDF.loc[index, 'LTeamID'])
    if startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']:
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] += GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Wins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FlipCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'WinCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'LoseCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Points'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PointsAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'OReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ORebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DRebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Stl'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'StlAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Blk'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'BlkAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PF'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PFAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
    # update losing team's up-to-the-minute season totals with results of game
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'CloseGames'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FlipCloseWins'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'WinCloseWins'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Points'] += rawGamesDF.loc[index, 'LScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PointsAllowed'] += rawGamesDF.loc[index, 'WScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] += rawGamesDF.loc[index, 'LFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] += rawGamesDF.loc[index, 'WFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] += rawGamesDF.loc[index, 'LFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] += rawGamesDF.loc[index, 'WFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] + rawGamesDF.loc[index, 'LFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] + rawGamesDF.loc[index, 'LFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] + rawGamesDF.loc[index, 'WFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] + rawGamesDF.loc[index, 'WFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] += rawGamesDF.loc[index, 'LFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] += rawGamesDF.loc[index, 'WFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] += rawGamesDF.loc[index, 'LFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] += rawGamesDF.loc[index, 'WFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] + rawGamesDF.loc[index, 'LFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] + rawGamesDF.loc[index, 'LFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] + rawGamesDF.loc[index, 'WFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] + rawGamesDF.loc[index, 'WFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] += rawGamesDF.loc[index, 'LFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] += rawGamesDF.loc[index, 'WFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] += rawGamesDF.loc[index, 'LFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] += rawGamesDF.loc[index, 'WFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] + rawGamesDF.loc[index, 'LFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] + rawGamesDF.loc[index, 'LFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] + rawGamesDF.loc[index, 'WFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] + rawGamesDF.loc[index, 'WFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'OReb'] += rawGamesDF.loc[index, 'LOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ORebAllowed'] += rawGamesDF.loc[index, 'WOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DReb'] += rawGamesDF.loc[index, 'LDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DRebAllowed'] += rawGamesDF.loc[index, 'WDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] += rawGamesDF.loc[index, 'LAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] += rawGamesDF.loc[index, 'WAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] += rawGamesDF.loc[index, 'LTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] += rawGamesDF.loc[index, 'WTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] + rawGamesDF.loc[index, 'LAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] + rawGamesDF.loc[index, 'LTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] + rawGamesDF.loc[index, 'WAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] + rawGamesDF.loc[index, 'WTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'LAst'] / rawGamesDF.loc[index, 'LTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'WAst'] / rawGamesDF.loc[index, 'WTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Stl'] += rawGamesDF.loc[index, 'LStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'StlAllowed'] += rawGamesDF.loc[index, 'WStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Blk'] += rawGamesDF.loc[index, 'LBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'BlkAllowed'] += rawGamesDF.loc[index, 'WBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PF'] += rawGamesDF.loc[index, 'LPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PFAllowed'] += rawGamesDF.loc[index, 'WPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ListOpponentsTeamID'].append(rawGamesDF.loc[index, 'WTeamID'])
    if startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']:
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] += GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Wins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FlipCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'WinCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'LoseCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Points'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PointsAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'OReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ORebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DRebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Stl'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'StlAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Blk'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'BlkAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PF'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PFAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])


Season: 2003    Current time: 2024-08-22 21:11:09
Season: 2003    Day Number: 10    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 11    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 12    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 13    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 14    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 15    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 16    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 17    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 18    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 19    Current Time: 2024-08-22 21:11:11
Season: 2003    Day Number: 20    Current Time: 2024-08-22 21:11:12
Season: 2003    Day Number: 21    Current Time: 2024-08-22 21:11:12
Season: 2003    Day Number: 22    Current Time: 2024-08-22 21:11:12
Season: 2003    Day Number: 23    Current Time: 2024-08-22 21:11:1

Team_Stats_Season_Totals_2003.csv written


In [ ]:
# generate training data

# initialize DataFrame to contain input data
emptyInputDF = pd.DataFrame(columns=[
    'Season',  # not a feature
    'DayNum',  # not a feature
    
    'TeamAWins',  # True if Team A wins, False if Team B wins
    'TeamID A',  # not a feature
    'TeamID B',  # not a feature

    # home-court advantage
    'HomeAdv A',  # +1 if Team A is Home, 0 if neutral site, -1 if Team A is Away

    # Team A season totals

    'NumGamesPlayedThisSeason A',  # the number of games Team A has played during the current season

    'WinPctThisSeason A',  # win percentage for Team A during the current season
    'WinPctLastSeason A',  # win percentage for Team A during the previous season

    'FlipCloseWinPctThisSeason A',  # win percentage for Team A during the current season (had the results of all of their close games been flipped)
    'FlipCloseWinPctLastSeason A',  # win percentage for Team A during the previous season (had the results of all of their close games been flipped)

    'WinCloseWinPctThisSeason A',  # win percentage for Team A during the current season (had they won all of their close games)
    'WinCloseWinPctLastSeason A',  # win percentage for Team A during the previous season (had they won all of their close games)

    'LoseCloseWinPctThisSeason A',  # win percentage for Team A during the current season (had they lost all of their close games)
    'LoseCloseWinPctLastSeason A',  # win percentage for Team A during the previous season (had they lost all of their close games)

    'PointsPerGameThisSeason A',  # average points scored by Team A per game during the current season
    'PointsPerGameLastSeason A',  # average points scored by Team A per game during the previous season

    'PointsAllowedPerGameThisSeason A',  # average points allowed by Team A per game during the current season
    'PointsAllowedPerGameLastSeason A',  # average points allowed by Team A per game during the previous season

    'PointsMarginPerGameThisSeason A',  # average difference between points scored by Team A and points allowed by Team A per game during the current season
    'PointsMarginPerGameLastSeason A',  # average difference between points scored by Team A and points allowed by Team A per game during the previous season

    'FGMPerGameThisSeason A',  # average number of field goals made by Team A per game during the current season
    'FGMPerGameLastSeason A',  # average number of field goals made by Team A per game during the previous season

    'FGMAllowedPerGameThisSeason A',  # average number of field goals allowed by Team A per game during the current season
    'FGMAllowedPerGameLastSeason A',  # average number of field goals allowed by Team A per game during the previous season

    'FGMMarginPerGameThisSeason A',  # average difference between field goals made by Team A and field goals allowed by Team A per game during the current season
    'FGMMarginPerGameLastSeason A',  # average difference between field goals made by Team A and field goals allowed by Team A per game during the previous season

    'FGAPerGameThisSeason A',  # average number of field goals attempted by Team A per game during the current season
    'FGAPerGameLastSeason A',  # average number of field goals attempted by Team A per game during the previous season

    'FGAAllowedPerGameThisSeason A',  # average number of field goal attempts allowed by Team A per game during the current season
    'FGAAllowedPerGameLastSeason A',  # average number of field goal attempts allowed by Team A per game during the previous season

    'FGAMarginPerGameThisSeason A',  # average difference between number of field goals attempted by Team A and number of field goal attempts allowed by Team A per game during the current season
    'FGAMarginPerGameLastSeason A',  # average difference between number of field goals attempted by Team A and number of field goal attempts allowed by Team A per game during the previous season

    'FGPctThisSeason A',  # Team A's field goal percentage during the current season
    'FGPctLastSeason A',  # Team A's field goal percentage during the previous season

    'FGPctAllowedThisSeason A',  # Team A's field goal percentage allowed during the current season
    'FGPctAllowedLastSeason A',  # Team A's field goal percentage allowed during the previous season

    'FGPctMarginThisSeason A',  # difference between Team A's field goal percentage and Team A's field goal percentage allowed during the current season
    'FGPctMarginLastSeason A',  # difference between Team A's field goal percentage and Team A's field goal percentage allowed during the previous season

    'FGM3PerGameThisSeason A',  # average number of 3-point field goals made by Team A per game during the current season
    'FGM3PerGameLastSeason A',  # average number of 3-point field goals made by Team A per game during the previous season

    'FGM3AllowedPerGameThisSeason A',  # average number of 3-point field goals allowed by Team A per game during the current season
    'FGM3AllowedPerGameLastSeason A',  # average number of 3-point field goals allowed by Team A per game during the previous season

    'FGM3MarginPerGameThisSeason A',  # average difference between number of 3-point field goals made by Team A and number of 3-point field goals allowed by Team A per game during the current season
    'FGM3MarginPerGameLastSeason A',  # average difference between number of 3-point field goals made by Team A and number of 3-point field goals allowed by Team A per game during the previous season

    'FGA3PerGameThisSeason A',  # average number of 3-point field goal attempts made by Team A per game during the current season
    'FGA3PerGameLastSeason A',  # average number of 3-point field goal attempts made by Team A per game during the previous season

    'FGA3AllowedPerGameThisSeason A',  # average number of 3-point field goal attempts allowed by Team A per game during the current season
    'FGA3AllowedPerGameLastSeason A',  # average number of 3-point field goal attempts allowed by Team A per game during the previous season

    'FGA3MarginPerGameThisSeason A',  # average difference between number of 3-point field goals attempted by Team A and number of 3-point field goal attempts allowed by Team A per game during the current season
    'FGA3MarginPerGameLastSeason A',  # average difference between number of 3-point field goals attempted by Team A and number of 3-point field goal attempts allowed by Team A per game during the previous season

    'FG3PctThisSeason A',  # Team A's 3-point field goal percentage during the current season
    'FG3PctLastSeason A',  # Team A's 3-point field goal percentage during the previous season

    'FG3PctAllowedThisSeason A',  # Team A's 3-point field goal percentage allowed during the current season
    'FG3PctAllowedLastSeason A',  # Team A's 3-point field goal percentage allowed during the previous season

    'FG3PctMarginThisSeason A',  # difference between Team A's 3-point field goal percentage and Team A's 3-point field goal percentage allowed during the current season
    'FG3PctMarginLastSeason A',  # difference between Team A's 3-point field goal percentage and Team A's 3-point field goal percentage allowed during the previous season

    'FTMPerGameThisSeason A',  # average number of free-throws made by Team A per game during the current season
    'FTMPerGameLastSeason A',  # average number of free-throws made by Team A per game during the previous season

    'FTMAllowedPerGameThisSeason A',  # average number of successful free-throws allowed by Team A per game during the current season
    'FTMAllowedPerGameLastSeason A',  # average number of successful free-throws allowed by Team A per game during the previous season

    'FTMMarginPerGameThisSeason A',  # average difference between number of free-throws made by Team A and number of successful free-throws allowed by Team A per game during the current season
    'FTMMarginPerGameLastSeason A',  # average difference between number of free-throws made by Team A and number of successful free-throws allowed by Team A per game during the previous season

    'FTAPerGameThisSeason A',  # average number of free-throws attempted by Team A per game during the current season
    'FTAPerGameLastSeason A',  # average number of free-throws attempted by Team A per game during the previous season

    'FTAAllowedPerGameThisSeason A',  # average number of free-throw attempts allowed by Team A per game during the current season
    'FTAAllowedPerGameLastSeason A',  # average number of free-throw attempts allowed by Team A per game during the previous season

    'FTAMarginPerGameThisSeason A',  # average difference between number of free-throws attempted by Team A and number of free-throw attempts allowed by Team A per game during the current season
    'FTAMarginPerGameLastSeason A',  # average difference between number of free-throws attempted by Team A and number of free-throw attempts allowed by Team A per game during the previous season

    'FTPctThisSeason A',  # Team A's free-throw percentage during the current season
    'FTPctLastSeason A',  # Team A's free-throw percentage during the previous season

    'FTPctAllowedThisSeason A',  # Team A's free-throw percentage allowed during the current season
    'FTPctAllowedLastSeason A',  # Team A's free-throw percentage allowed during the previous season

    'FTPctMarginThisSeason A',  # difference between Team A's free-throw percentage and Team A's free-throw percentage allowed during the current season
    'FTPctMarginLastSeason A',  # difference between Team A's free-throw percentage and Team A's free-throw percentage allowed during the previous season

    'RebPerGameThisSeason A',  # average number of rebounds by Team A per game during the current season
    'RebPerGameLastSeason A',  # average number of rebounds by Team A per game during the previous season

    'RebAllowedPerGameThisSeason A',  # average number of rebounds allowed by Team A per game during the current season
    'RebAllowedPerGameLastSeason A',  # average number of rebounds allowed by Team A per game during the previous season

    'RebMarginPerGameThisSeason A',  # average difference between number of rebounds by Team A and number of rebounds allowed by Team A per game during the current season
    'RebMarginPerGameLastSeason A',  # average difference between number of rebounds by Team A and number of rebounds allowed by Team A per game during the previous season

    'ORebPerGameThisSeason A',  # average number of offensive rebounds by Team A per game during the current season
    'ORebPerGameLastSeason A',  # average number of offensive rebounds by Team A per game during the previous season

    'ORebAllowedPerGameThisSeason A',  # average number of offensive rebounds allowed by Team A per game during the current season
    'ORebAllowedPerGameLastSeason A',  # average number of offensive rebounds allowed by Team A per game during the previous season

    'ORebMarginPerGameThisSeason A',  # average difference between number of offensive rebounds by Team A and number of rebounds allowed by Team A per game during the current season
    'ORebMarginPerGameLastSeason A',  # average difference between number of offensive rebounds by Team A and number of rebounds allowed by Team A per game during the previous season

    'DRebPerGameThisSeason A',  # average number of defensive rebounds by Team A per game during the current season
    'DRebPerGameLastSeason A',  # average number of defensive rebounds by Team A per game during the previous season

    'DRebAllowedPerGameThisSeason A',  # average number of defensive rebounds allowed by Team A per game during the current season
    'DRebAllowedPerGameLastSeason A',  # average number of defensive rebounds allowed by Team A per game during the previous season

    'DRebMarginPerGameThisSeason A',  # average difference between number of defensive rebounds by Team A and number of rebounds allowed by Team A per game during the current season
    'DRebMarginPerGameLastSeason A',  # average difference between number of defensive rebounds by Team A and number of rebounds allowed by Team A per game during the previous season

    'AstPerGameThisSeason A',  # average number of assists by Team A per game during the current season
    'AstPerGameLastSeason A',  # average number of assists by Team A per game during the previous season

    'AstAllowedPerGameThisSeason A',  # average number of assists allowed by Team A per game during the current season
    'AstAllowedPerGameLastSeason A',  # average number of assists allowed by Team A per game during the previous season

    'AstMarginPerGameThisSeason A',  # average difference between number of assists by Team A and number of assists allowed by Team A per game during the current season
    'AstMarginPerGameLastSeason A',  # average difference between number of assists by Team A and number of assists allowed by Team A per game during the previous season

    'TOPerGameThisSeason A',  # average number of turnovers by Team A per game during the current season
    'TOPerGameLastSeason A',  # average number of turnovers by Team A per game during the previous season

    'TOAllowedPerGameThisSeason A',  # average number of turnovers forced by Team A per game during the current season
    'TOAllowedPerGameLastSeason A',  # average number of turnovers forced by Team A per game during the previous season

    'TOMarginPerGameThisSeason A',  # average difference between number of turnovers by Team A and number of turnovers forced by Team A per game during the current season
    'TOMarginPerGameLastSeason A',  # average difference between number of turnovers by Team A and number of turnovers forced by Team A per game during the previous season

    'AvgAst/TORatioThisSeason A',  # Team A's season total assists divided by Team A's season total turnovers during the current season
    'AvgAst/TORatioLastSeason A',  # Team A's season total assists divided by Team A's season total turnovers during the previous season

    'AvgAst/TORatioAllowedThisSeason A',  # Team A's season total assists allowed divided by Team A's season total turnovers forced during the current season
    'AvgAst/TORatioAllowedLastSeason A',  # Team A's season total assists allowed divided by Team A's season total turnovers forced during the previous season

    'AvgAst/TORatioMarginThisSeason A',  # (Team A's season total assists divided by Team A's season total turnovers) minus (Team A's season total assists allowed divided by Team A's season total turnovers forced) during the current season
    'AvgAst/TORatioMarginLastSeason A',  # (Team A's season total assists divided by Team A's season total turnovers) minus (Team A's season total assists allowed divided by Team A's season total turnovers forced) during the previous season

    'AvgAst/TORatio(alt)ThisSeason A',  # average of Team A's assist/turnover ratios for each game Team A has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)LastSeason A',  # average of Team A's assist/turnover ratios for each game Team A played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'AvgAst/TORatio(alt)AllowedThisSeason A',  # average of Team A's opponent's assist/turnover ratios for each game Team A has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)AllowedLastSeason A',  # average of Team A's opponent's assist/turnover ratios for each game Team A has played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'AvgAst/TORatio(alt)MarginThisSeason A',  # average of the differences between Team A's assist/turnover ratio and Team A's opponent's assist/turnover ratio for each game Team A has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)MarginLastSeason A',  # average of the differences between Team A's assist/turnover ratio and Team A's opponent's assist/turnover ratio for each game Team A played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'StlPerGameThisSeason A',  # average number of steals by Team A per game during the current season
    'StlPerGameLastSeason A',  # average number of steals by Team A per game during the previous season

    'StlAllowedPerGameThisSeason A',  # average number of steals allowed by Team A per game during the current season
    'StlAllowedPerGameLastSeason A',  # average number of steals allowed by Team A per game during the previous season

    'StlMarginPerGameThisSeason A',  # average difference between number of steals by Team A and number of steals allowed by Team A per game during the current season
    'StlMarginPerGameLastSeason A',  # average difference between number of steals by Team A and number of steals allowed by Team A per game during the previous season

    'BlkPerGameThisSeason A',  # average number of blocks by Team A per game during the current season
    'BlkPerGameLastSeason A',  # average number of blocks by Team A per game during the previous season

    'BlkAllowedPerGameThisSeason A',  # average number of blocks allowed by Team A per game during the current season
    'BlkAllowedPerGameLastSeason A',  # average number of blocks allowed by Team A per game during the previous season

    'BlkMarginPerGameThisSeason A',  # average difference between number of blocks by Team A and number of blocks allowed by Team A per game during the current season
    'BlkMarginPerGameLastSeason A',  # average difference between number of blocks by Team A and number of blocks allowed by Team A per game during the previous season

    'PFPerGameThisSeason A',  # average number of fouls committed by Team A per game during the current season
    'PFPerGameLastSeason A',  # average number of fouls committed by Team A per game during the previous season

    'PFAllowedPerGameThisSeason A',  # average number of fouls absorbed by Team A per game during the current season
    'PFAllowedPerGameLastSeason A',  # average number of fouls absorbed by Team A per game during the previous season

    'PFMarginPerGameThisSeason A',  # average difference between number of fouls committed by Team A and number of fouls absorbed by Team A per game during the current season
    'PFMarginPerGameLastSeason A',  # average difference between number of fouls committed by Team A and number of fouls absorbed by Team A per game during the previous season

    # Team A strength of schedule

    'CurrentSoSWeightedAvgOppWinPctThenThisSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the current season
    'PastSoSWeightedAvgOppWinPctThenLastSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the
    'CurrentSoSAvgOppWinPctNowThisSeason A',  # average up-to-date win percentage (counting only the current season) of the teams which Team A has played during the current season
    'CurrentSoSAvgOppWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A has played during the current season
    'PastSoSAvgOppWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A played during the previous season

    'CurrentSoSWeightedAvgOppFlipCloseWinPctThenThisSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the current season (had the results of all of their close games been flipped)
    'PastSoSWeightedAvgOppFlipCloseWinPctThenLastSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the previous season (had the results of all of their close games been flipped)
    'CurrentSoSAvgOppFlipCloseWinPctNowThisSeason A',  # average up-to-date win percentage (counting only the current season) of the teams which Team A has played during the current season (had the results of all of their close games been flipped)
    'CurrentSoSAvgOppFlipCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A has played during the current season (had the results of all of their close games been flipped)
    'PastSoSAvgOppFlipCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A played during the previous season (had the results of all of their close games been flipped)

    'CurrentSoSWeightedAvgOppWinCloseWinPctThenThisSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the current season (had they won all of their close games)
    'PastSoSWeightedAvgOppWinCloseWinPctThenLastSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the previous season (had they won all of their close games)
    'CurrentSoSAvgOppWinCloseWinPctNowThisSeason A',  # average up-to-date win percentage (counting only the current season) of the teams which Team A has played during the current season (had they won all of their close games)
    'CurrentSoSAvgOppWinCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A has played during the current season (had they won all of their close games)
    'PastSoSAvgOppWinCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A played during the previous season (had they won all of their close games)

    'CurrentSoSWeightedAvgOppLoseCloseWinPctThenThisSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the current season (had they lost all of their close games)
    'PastSoSWeightedAvgOppLoseCloseWinPctThenLastSeason A',  # average win percentage of Team A's opponents at the time when they played Team A, weighted by number of games played by opponent going into the game against Team A, during the previous season (had they lost all of their close games)
    'CurrentSoSAvgOppLoseCloseWinPctNowThisSeason A',  # average up-to-date win percentage (counting only the current season) of the teams which Team A has played during the current season (had they lost all of their close games)
    'CurrentSoSAvgOppLoseCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A has played during the current season (had they lost all of their close games)
    'PastSoSAvgOppLoseCloseWinPctNowLastSeason A',  # average end-of-season win percentage (counting only the previous season) of the teams which Team A played during the previous season (had they lost all of their close games)

    'CurrentSoSWeightedAvgOppPointsPerGameThenThisSeason A',  # average of Team A's current-season opponents' average points scored per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average points scored per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average points scored per game during the current season
    'CurrentSoSAvgOppPointsPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average points scored per game during the previous season
    'PastSoSAvgOppPointsPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average points scored per game during the previous season

    'CurrentSoSWeightedAvgOppPointsAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average points allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average points allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average points allowed per game during the current season
    'CurrentSoSAvgOppPointsAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average points allowed per game during the previous season
    'PastSoSAvgOppPointsAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average points allowed per game during the previous season

    'CurrentSoSWeightedAvgOppPointsMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between points scored and points allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between points scored and points allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between points scored and points allowed per game during the current season
    'CurrentSoSAvgOppPointsMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between points scored and points allowed per game during the previous season
    'PastSoSAvgOppPointsMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between points scored and points allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGMPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of field goals made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of field goals made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of field goals made per game during the current season
    'CurrentSoSAvgOppFGMPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of field goals made per game during the previous season
    'PastSoSAvgOppFGMPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of field goals made per game during the previous season

    'CurrentSoSWeightedAvgOppFGMAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of field goals allowed per game during the current season
    'CurrentSoSAvgOppFGMAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of field goals allowed per game during the previous season
    'PastSoSAvgOppFGMAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGMMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of field goals made and number of field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of field goals made and number of field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of field goals made and number of field goals allowed per game during the current season
    'CurrentSoSAvgOppFGMMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of field goals made and number of field goals allowed per game during the previous season
    'PastSoSAvgOppFGMMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of field goals made and number of field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGAPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of field goal attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of field goal attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of field goal attempts per game during the current season
    'CurrentSoSAvgOppFGAPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of field goal attempts per game during the previous season
    'PastSoSAvgOppFGAPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of field goal attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFGAAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGAAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGAAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGAMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of field goal attempts and number of field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of field goal attempts and number of field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of field goal attempts and number of field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGAMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of field goal attempts and number of field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGAMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of field goal attempts and number of field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGPctThenThisSeason A',  # average of Team A's current-season opponents' season-long field goal percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctThenLastSeason A',  # average of Team A's previous-season opponents' season-long field goal percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long field goal percentage during the current season
    'CurrentSoSAvgOppFGPctNowLastSeason A',  # average of Team A's current-season opponents' season-long field goal percentage during the previous season
    'PastSoSAvgOppFGPctNowLastSeason A',  # average of Team A's previous-season opponents' season-long field goal percentage during the previous season

    'CurrentSoSWeightedAvgOppFGPctAllowedThenThisSeason A',  # average of Team A's current-season opponents' season-long field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctAllowedThenLastSeason A',  # average of Team A's previous-season opponents' season-long field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctAllowedNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long field goal percentage allowed during the current season
    'CurrentSoSAvgOppFGPctAllowedNowLastSeason A',  # average of Team A's current-season opponents' season-long field goal percentage allowed during the previous season
    'PastSoSAvgOppFGPctAllowedNowLastSeason A',  # average of Team A's previous-season opponents' season-long field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFGPctMarginThenThisSeason A',  # average of Team A's current-season opponents' differences between season-long field goal percentage and season-long field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctMarginThenLastSeason A',  # average of Team A's previous-season opponents' differences between previous-season-long field goal percentage and previous-season-long field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctMarginNowThisSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long field goal percentage and season-long field goal percentage allowed during the current season
    'CurrentSoSAvgOppFGPctMarginNowLastSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long field goal percentage and season-long field goal percentage allowed during the previous season
    'PastSoSAvgOppFGPctMarginNowLastSeason A',  # average of Team A's previous-season opponents' differences between season-long field goal percentage and season-long field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFGM3PerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of 3-point field goals made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3PerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of 3-point field goals made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3PerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of 3-point field goals made per game during the current season
    'CurrentSoSAvgOppFGM3PerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of 3-point field goals made per game during the previous season
    'PastSoSAvgOppFGM3PerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of 3-point field goals made per game during the previous season

    'CurrentSoSWeightedAvgOppFGM3AllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of 3-point field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3AllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of 3-point field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3AllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of 3-point field goals allowed per game during the current season
    'CurrentSoSAvgOppFGM3AllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of 3-point field goals allowed per game during the previous season
    'PastSoSAvgOppFGM3AllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of 3-point field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGM3MarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of 3-point field goals made and number of 3-point field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3MarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of 3-point field goals made and number of 3-point field goals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3MarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the current season
    'CurrentSoSAvgOppFGM3MarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the previous season
    'PastSoSAvgOppFGM3MarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3PerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of 3-point field goal attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3PerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of 3-point field goal attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3PerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of 3-point field goal attempts per game during the current season
    'CurrentSoSAvgOppFGA3PerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of 3-point field goal attempts per game during the previous season
    'PastSoSAvgOppFGA3PerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of 3-point field goal attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3AllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3AllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3AllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of 3-point field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGA3AllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of 3-point field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGA3AllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of 3-point field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3MarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3MarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3MarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGA3MarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGA3MarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFG3PctThenThisSeason A',  # average of Team A's current-season opponents' season-long 3-point field goal percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctThenLastSeason A',  # average of Team A's previous-season opponents' season-long 3-point field goal percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long 3-point field goal percentage during the current season
    'CurrentSoSAvgOppFG3PctNowLastSeason A',  # average of Team A's current-season opponents' season-long 3-point field goal percentage during the previous season
    'PastSoSAvgOppFG3PctNowLastSeason A',  # average of Team A's previous-season opponents' season-long 3-point field goal percentage during the previous season

    'CurrentSoSWeightedAvgOppFG3PctAllowedThenThisSeason A',  # average of Team A's current-season opponents' season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctAllowedThenLastSeason A',  # average of Team A's previous-season opponents' season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctAllowedNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long 3-point field goal percentage allowed during the current season
    'CurrentSoSAvgOppFG3PctAllowedNowLastSeason A',  # average of Team A's current-season opponents' season-long 3-point field goal percentage allowed during the previous season
    'PastSoSAvgOppFG3PctAllowedNowLastSeason A',  # average of Team A's previous-season opponents' season-long 3-point field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFG3PctMarginThenThisSeason A',  # average of Team A's current-season opponents' differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctMarginThenLastSeason A',  # average of Team A's previous-season opponents' differences between previous-season-long 3-point field goal percentage and previous-season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctMarginNowThisSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the current season
    'CurrentSoSAvgOppFG3PctMarginNowLastSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the previous season
    'PastSoSAvgOppFG3PctMarginNowLastSeason A',  # average of Team A's previous-season opponents' differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFTMPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of free-throws made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of free-throws made per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of free-throws made per game during the current season
    'CurrentSoSAvgOppFTMPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of free-throws made per game during the previous season
    'PastSoSAvgOppFTMPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of free-throws made per game during the previous season

    'CurrentSoSWeightedAvgOppFTMAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of successful free-throws allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of successful free-throws allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of successful free-throws allowed per game during the current season
    'CurrentSoSAvgOppFTMAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of successful free-throws allowed per game during the previous season
    'PastSoSAvgOppFTMAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of successful free-throws allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTMMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of free-throws made and number of successful free-throws allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of free-throws made and number of successful free-throws allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of free-throws made and number of successful free-throws allowed per game during the current season
    'CurrentSoSAvgOppFTMMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of free-throws made and number of successful free-throws allowed per game during the previous season
    'PastSoSAvgOppFTMMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of free-throws made and number of successful free-throws allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTAPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of free-throw attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of free-throw attempts per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of free-throw attempts per game during the current season
    'CurrentSoSAvgOppFTAPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of free-throw attempts per game during the previous season
    'PastSoSAvgOppFTAPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of free-throw attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFTAAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average number of free-throw attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average number of free-throw attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average number of free-throw attempts allowed per game during the current season
    'CurrentSoSAvgOppFTAAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average number of free-throw attempts allowed per game during the previous season
    'PastSoSAvgOppFTAAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average number of free-throw attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTAMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between number of free-throw attempts and number of free-throw attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between number of free-throw attempts and number of free-throw attempts allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the current season
    'CurrentSoSAvgOppFTAMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the previous season
    'PastSoSAvgOppFTAMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTPctThenThisSeason A',  # average of Team A's current-season opponents' season-long free-throw percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctThenLastSeason A',  # average of Team A's previous-season opponents' season-long free-throw percentage, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long free-throw percentage during the current season
    'CurrentSoSAvgOppFTPctNowLastSeason A',  # average of Team A's current-season opponents' season-long free-throw percentage during the previous season
    'PastSoSAvgOppFTPctNowLastSeason A',  # average of Team A's previous-season opponents' season-long free-throw percentage during the previous season

    'CurrentSoSWeightedAvgOppFTPctAllowedThenThisSeason A',  # average of Team A's current-season opponents' season-long free-throw percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctAllowedThenLastSeason A',  # average of Team A's previous-season opponents' season-long free-throw percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctAllowedNowThisSeason A',  # average of Team A's current-season opponents' up-to-date season-long free-throw percentage allowed during the current season
    'CurrentSoSAvgOppFTPctAllowedNowLastSeason A',  # average of Team A's current-season opponents' season-long free-throw percentage allowed during the previous season
    'PastSoSAvgOppFTPctAllowedNowLastSeason A',  # average of Team A's previous-season opponents' season-long free-throw percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFTPctMarginThenThisSeason A',  # average of Team A's current-season opponents' differences between season-long free-throw percentage and season-long free-throw percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctMarginThenLastSeason A',  # average of Team A's previous-season opponents' differences between previous-season-long free-throw percentage and previous-season-long free-throw percentage allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctMarginNowThisSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long free-throw percentage and season-long free-throw percentage allowed during the current season
    'CurrentSoSAvgOppFTPctMarginNowLastSeason A',  # average of Team A's current-season opponents' up-to-date differences between season-long free-throw percentage and season-long free-throw percentage allowed during the previous season
    'PastSoSAvgOppFTPctMarginNowLastSeason A',  # average of Team A's previous-season opponents' differences between season-long free-throw percentage and season-long free-throw percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppRebPerGameThenThisSeason A',  # average of Team A's current-season opponents' average rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average rebounds per game during the current season
    'CurrentSoSAvgOppRebPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average rebounds per game during the previous season
    'PastSoSAvgOppRebPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppRebAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average rebounds allowed per game during the current season
    'CurrentSoSAvgOppRebAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average rebounds allowed per game during the previous season
    'PastSoSAvgOppRebAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppRebMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between rebounds and rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between rebounds and rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between rebounds and rebounds allowed per game during the current season
    'CurrentSoSAvgOppRebMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between rebounds and rebounds allowed per game during the previous season
    'PastSoSAvgOppRebMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between rebounds and rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppORebPerGameThenThisSeason A',  # average of Team A's current-season opponents' average offensive rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average offensive rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average offensive rebounds per game during the current season
    'CurrentSoSAvgOppORebPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average offensive rebounds per game during the previous season
    'PastSoSAvgOppORebPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average offensive rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppORebAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average offensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average offensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average offensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppORebAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average offensive rebounds allowed per game during the previous season
    'PastSoSAvgOppORebAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average offensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppORebMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between offensive rebounds and offensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between offensive rebounds and offensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between offensive rebounds and offensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppORebMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between offensive rebounds and offensive rebounds allowed per game during the previous season
    'PastSoSAvgOppORebMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between offensive rebounds and offensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppDRebPerGameThenThisSeason A',  # average of Team A's current-season opponents' average defensive rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average defensive rebounds per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average defensive rebounds per game during the current season
    'CurrentSoSAvgOppDRebPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average defensive rebounds per game during the previous season
    'PastSoSAvgOppDRebPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average defensive rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppDRebAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average defensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average defensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average defensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppDRebAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average defensive rebounds allowed per game during the previous season
    'PastSoSAvgOppDRebAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average defensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppDRebMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between defensive rebounds and defensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between defensive rebounds and defensive rebounds allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between defensive rebounds and defensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppDRebMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between defensive rebounds and defensive rebounds allowed per game during the previous season
    'PastSoSAvgOppDRebMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between defensive rebounds and defensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppAstPerGameThenThisSeason A',  # average of Team A's current-season opponents' average assists per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average assists per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average assists per game during the current season
    'CurrentSoSAvgOppAstPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average assists per game during the previous season
    'PastSoSAvgOppAstPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average assists per game during the previous season

    'CurrentSoSWeightedAvgOppAstAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average assists allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average assists allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average assists allowed per game during the current season
    'CurrentSoSAvgOppAstAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average assists allowed per game during the previous season
    'PastSoSAvgOppAstAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average assists allowed per game during the previous season

    'CurrentSoSWeightedAvgOppAstMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between assists and assists allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between assists and assists allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between assists and assists allowed per game during the current season
    'CurrentSoSAvgOppAstMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between assists and assists allowed per game during the previous season
    'PastSoSAvgOppAstMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between assists and assists allowed per game during the previous season

    'CurrentSoSWeightedAvgOppTOPerGameThenThisSeason A',  # average of Team A's current-season opponents' average turnovers per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average turnovers per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average turnovers per game during the current season
    'CurrentSoSAvgOppTOPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average turnovers per game during the previous season
    'PastSoSAvgOppTOPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average turnovers per game during the previous season

    'CurrentSoSWeightedAvgOppTOAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average turnovers forced per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average turnovers forced per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average turnovers forced per game during the current season
    'CurrentSoSAvgOppTOAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average turnovers forced per game during the previous season
    'PastSoSAvgOppTOAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average turnovers forced per game during the previous season

    'CurrentSoSWeightedAvgOppTOMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between turnovers and turnovers forced per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between turnovers and turnovers forced per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between turnovers and turnovers forced per game during the current season
    'CurrentSoSAvgOppTOMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between turnovers and turnovers forced per game during the previous season
    'PastSoSAvgOppTOMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between turnovers and turnovers forced per game during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioThenThisSeason A',  # average of Team A's current-season opponents' current-season-long assist/turnover ratios (total assists / total turnovers), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long assist/turnover ratios (total assists / total turnovers), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioNowThisSeason A',  # average of Team A's current-season opponents' up-to-date assist/turnover ratios (total assists / total turnovers) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioNowLastSeason A',  # average of Team A's current-season opponents' season-long assist/turnover ratios (total assists / total turnovers) during the previous season
    'PastSoSAvgOppAvgAst/TORatioNowLastSeason A',  # average of Team A's previous-season opponents' season-long assist/turnover ratios (total assists / total turnovers) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioAllowedThenThisSeason A',  # average of Team A's current-season opponents' current-season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioAllowedThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioAllowedNowThisSeason A',  # average of Team A's current-season opponents' up-to-date assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioAllowedNowLastSeason A',  # average of Team A's current-season opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the previous season
    'PastSoSAvgOppAvgAst/TORatioAllowedNowLastSeason A',  # average of Team A's previous-season opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioMarginThenThisSeason A',  # average of Team A's current-season opponents' current-season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioMarginThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioMarginNowThisSeason A',  # average of Team A's current-season opponents' up-to-date differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioMarginNowLastSeason A',  # average of Team A's current-season opponents' season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the previous season
    'PastSoSAvgOppAvgAst/TORatioMarginNowLastSeason A',  # average of Team A's previous-season opponents' season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)ThenThisSeason A',  # average of Team A's current-season opponents' current-season-long assist/turnover ratios, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)ThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long assist/turnover ratios, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)NowThisSeason A',  # average of Team A's current-season opponents' up-to-date assist/turnover ratios during the current season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)NowLastSeason A',  # average of Team A's current-season opponents' season-long assist/turnover ratios during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSAvgOppAvgAst/TORatio(alt)NowLastSeason A',  # average of Team A's previous-season opponents' season-long assist/turnover ratios during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenThisSeason A',  # average of Team A's current-season opponents' current-season-long assist/turnover ratios allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long assist/turnover ratios allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowThisSeason A',  # average of Team A's current-season opponents' up-to-date assist/turnover ratios allowed during the current season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason A',  # average of Team A's current-season opponents' season-long assist/turnover ratios allowed during the previous season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'PastSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason A',  # average of Team A's previous-season opponents' season-long assist/turnover ratios allowed during the previous season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenThisSeason A',  # average of Team A's current-season opponents' current-season-long differences between assist/turnover ratio and assist/turnover ratio allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenLastSeason A',  # average of Team A's previous-season opponents' previous-season-long differences between assist/turnover ratio and assist/turnover ratio allowed, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowThisSeason A',  # average of Team A's current-season opponents' up-to-date differences between assist/turnover ratio and assist/turnover ratio allowed during the current season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason A',  # average of Team A's current-season opponents' season-long differences between assist/turnover ratio and assist/turnover ratio allowed during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason A',  # average of Team A's previous-season opponents' season-long differences between assist/turnover ratio and assist/turnover ratio allowed during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'CurrentSoSWeightedAvgOppStlPerGameThenThisSeason A',  # average of Team A's current-season opponents' average steals per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average steals per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average steals per game during the current season
    'CurrentSoSAvgOppStlPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average steals per game during the previous season
    'PastSoSAvgOppStlPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average steals per game during the previous season

    'CurrentSoSWeightedAvgOppStlAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average steals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average steals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average steals allowed per game during the current season
    'CurrentSoSAvgOppStlAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average steals allowed per game during the previous season
    'PastSoSAvgOppStlAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average steals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppStlMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between steals and steals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between steals and steals allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between steals and steals allowed per game during the current season
    'CurrentSoSAvgOppStlMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between steals and steals allowed per game during the previous season
    'PastSoSAvgOppStlMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between steals and steals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppBlkPerGameThenThisSeason A',  # average of Team A's current-season opponents' average blocks per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average blocks per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average blocks per game during the current season
    'CurrentSoSAvgOppBlkPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average blocks per game during the previous season
    'PastSoSAvgOppBlkPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average blocks per game during the previous season

    'CurrentSoSWeightedAvgOppBlkAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average blocks allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average blocks allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average blocks allowed per game during the current season
    'CurrentSoSAvgOppBlkAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average blocks allowed per game during the previous season
    'PastSoSAvgOppBlkAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average blocks allowed per game during the previous season

    'CurrentSoSWeightedAvgOppBlkMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between blocks and blocks allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between blocks and blocks allowed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between blocks and blocks allowed per game during the current season
    'CurrentSoSAvgOppBlkMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between blocks and blocks allowed per game during the previous season
    'PastSoSAvgOppBlkMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between blocks and blocks allowed per game during the previous season

    'CurrentSoSWeightedAvgOppPFPerGameThenThisSeason A',  # average of Team A's current-season opponents' average fouls committed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average fouls committed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average fouls committed per game during the current season
    'CurrentSoSAvgOppPFPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average fouls committed per game during the previous season
    'PastSoSAvgOppPFPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average fouls committed per game during the previous season

    'CurrentSoSWeightedAvgOppPFAllowedPerGameThenThisSeason A',  # average of Team A's current-season opponents' average fouls absorbed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFAllowedPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average fouls absorbed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFAllowedPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average fouls absorbed per game during the current season
    'CurrentSoSAvgOppPFAllowedPerGameNowLastSeason A',  # average of Team A's current-season opponents' season average fouls absorbed per game during the previous season
    'PastSoSAvgOppPFAllowedPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average fouls absorbed per game during the previous season

    'CurrentSoSWeightedAvgOppPFMarginPerGameThenThisSeason A',  # average of Team A's current-season opponents' average difference between fouls committed and fouls absorbed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFMarginPerGameThenLastSeason A',  # average of Team A's previous-season opponents' average difference between fouls committed and fouls absorbed per game, recorded immediately prior to playing Team A, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFMarginPerGameNowThisSeason A',  # average of Team A's current-season opponents' up-to-date average difference between fouls committed and fouls absorbed per game during the current season
    'CurrentSoSAvgOppPFMarginPerGameNowLastSeason A',  # average of Team A's current-season opponents' season-long average difference between fouls committed and fouls absorbed per game during the previous season
    'PastSoSAvgOppPFMarginPerGameNowLastSeason A',  # average of Team A's previous-season opponents' season average difference between fouls committed and fouls absorbed per game during the previous season

    # Team B season totals

    'NumGamesPlayedThisSeason B',  # the number of games Team B has played during the current season

    'WinPctThisSeason B',  # win percentage for Team B during the current season
    'WinPctLastSeason B',  # win percentage for Team B during the previous season

    'FlipCloseWinPctThisSeason B',  # win percentage for Team B during the current season (had the results of all of their close games been flipped)
    'FlipCloseWinPctLastSeason B',  # win percentage for Team B during the previous season (had the results of all of their close games been flipped)

    'WinCloseWinPctThisSeason B',  # win percentage for Team B during the current season (had they won all of their close games)
    'WinCloseWinPctLastSeason B',  # win percentage for Team B during the previous season (had they won all of their close games)

    'LoseCloseWinPctThisSeason B',  # win percentage for Team B during the current season (had they lost all of their close games)
    'LoseCloseWinPctLastSeason B',  # win percentage for Team B during the previous season (had they lost all of their close games)

    'PointsPerGameThisSeason B',  # average points scored by Team B per game during the current season
    'PointsPerGameLastSeason B',  # average points scored by Team B per game during the previous season

    'PointsAllowedPerGameThisSeason B',  # average points allowed by Team B per game during the current season
    'PointsAllowedPerGameLastSeason B',  # average points allowed by Team B per game during the previous season

    'PointsMarginPerGameThisSeason B',  # average difference between points scored by Team B and points allowed by Team B per game during the current season
    'PointsMarginPerGameLastSeason B',  # average difference between points scored by Team B and points allowed by Team B per game during the previous season

    'FGMPerGameThisSeason B',  # average number of field goals made by Team B per game during the current season
    'FGMPerGameLastSeason B',  # average number of field goals made by Team B per game during the previous season

    'FGMAllowedPerGameThisSeason B',  # average number of field goals allowed by Team B per game during the current season
    'FGMAllowedPerGameLastSeason B',  # average number of field goals allowed by Team B per game during the previous season

    'FGMMarginPerGameThisSeason B',  # average difference between field goals made by Team B and field goals allowed by Team B per game during the current season
    'FGMMarginPerGameLastSeason B',  # average difference between field goals made by Team B and field goals allowed by Team B per game during the previous season

    'FGAPerGameThisSeason B',  # average number of field goals attempted by Team B per game during the current season
    'FGAPerGameLastSeason B',  # average number of field goals attempted by Team B per game during the previous season

    'FGAAllowedPerGameThisSeason B',  # average number of field goal attempts allowed by Team B per game during the current season
    'FGAAllowedPerGameLastSeason B',  # average number of field goal attempts allowed by Team B per game during the previous season

    'FGAMarginPerGameThisSeason B',  # average difference between number of field goals attempted by Team B and number of field goal attempts allowed by Team B per game during the current season
    'FGAMarginPerGameLastSeason B',  # average difference between number of field goals attempted by Team B and number of field goal attempts allowed by Team B per game during the previous season

    'FGPctThisSeason B',  # Team B's field goal percentage during the current season
    'FGPctLastSeason B',  # Team B's field goal percentage during the previous season

    'FGPctAllowedThisSeason B',  # Team B's field goal percentage allowed during the current season
    'FGPctAllowedLastSeason B',  # Team B's field goal percentage allowed during the previous season

    'FGPctMarginThisSeason B',  # difference between Team B's field goal percentage and Team B's field goal percentage allowed during the current season
    'FGPctMarginLastSeason B',  # difference between Team B's field goal percentage and Team B's field goal percentage allowed during the previous season

    'FGM3PerGameThisSeason B',  # average number of 3-point field goals made by Team B per game during the current season
    'FGM3PerGameLastSeason B',  # average number of 3-point field goals made by Team B per game during the previous season

    'FGM3AllowedPerGameThisSeason B',  # average number of 3-point field goals allowed by Team B per game during the current season
    'FGM3AllowedPerGameLastSeason B',  # average number of 3-point field goals allowed by Team B per game during the previous season

    'FGM3MarginPerGameThisSeason B',  # average difference between number of 3-point field goals made by Team B and number of 3-point field goals allowed by Team B per game during the current season
    'FGM3MarginPerGameLastSeason B',  # average difference between number of 3-point field goals made by Team B and number of 3-point field goals allowed by Team B per game during the previous season

    'FGA3PerGameThisSeason B',  # average number of 3-point field goal attempts made by Team B per game during the current season
    'FGA3PerGameLastSeason B',  # average number of 3-point field goal attempts made by Team B per game during the previous season

    'FGA3AllowedPerGameThisSeason B',  # average number of 3-point field goal attempts allowed by Team B per game during the current season
    'FGA3AllowedPerGameLastSeason B',  # average number of 3-point field goal attempts allowed by Team B per game during the previous season

    'FGA3MarginPerGameThisSeason B',  # average difference between number of 3-point field goals attempted by Team B and number of 3-point field goal attempts allowed by Team B per game during the current season
    'FGA3MarginPerGameLastSeason B',  # average difference between number of 3-point field goals attempted by Team B and number of 3-point field goal attempts allowed by Team B per game during the previous season

    'FG3PctThisSeason B',  # Team B's 3-point field goal percentage during the current season
    'FG3PctLastSeason B',  # Team B's 3-point field goal percentage during the previous season

    'FG3PctAllowedThisSeason B',  # Team B's 3-point field goal percentage allowed during the current season
    'FG3PctAllowedLastSeason B',  # Team B's 3-point field goal percentage allowed during the previous season

    'FG3PctMarginThisSeason B',  # difference between Team B's 3-point field goal percentage and Team B's 3-point field goal percentage allowed during the current season
    'FG3PctMarginLastSeason B',  # difference between Team B's 3-point field goal percentage and Team B's 3-point field goal percentage allowed during the previous season

    'FTMPerGameThisSeason B',  # average number of free-throws made by Team B per game during the current season
    'FTMPerGameLastSeason B',  # average number of free-throws made by Team B per game during the previous season

    'FTMAllowedPerGameThisSeason B',  # average number of successful free-throws allowed by Team B per game during the current season
    'FTMAllowedPerGameLastSeason B',  # average number of successful free-throws allowed by Team B per game during the previous season

    'FTMMarginPerGameThisSeason B',  # average difference between number of free-throws made by Team B and number of successful free-throws allowed by Team B per game during the current season
    'FTMMarginPerGameLastSeason B',  # average difference between number of free-throws made by Team B and number of successful free-throws allowed by Team B per game during the previous season

    'FTAPerGameThisSeason B',  # average number of free-throws attempted by Team B per game during the current season
    'FTAPerGameLastSeason B',  # average number of free-throws attempted by Team B per game during the previous season

    'FTAAllowedPerGameThisSeason B',  # average number of free-throw attempts allowed by Team B per game during the current season
    'FTAAllowedPerGameLastSeason B',  # average number of free-throw attempts allowed by Team B per game during the previous season

    'FTAMarginPerGameThisSeason B',  # average difference between number of free-throws attempted by Team B and number of free-throw attempts allowed by Team B per game during the current season
    'FTAMarginPerGameLastSeason B',  # average difference between number of free-throws attempted by Team B and number of free-throw attempts allowed by Team B per game during the previous season

    'FTPctThisSeason B',  # Team B's free-throw percentage during the current season
    'FTPctLastSeason B',  # Team B's free-throw percentage during the previous season

    'FTPctAllowedThisSeason B',  # Team B's free-throw percentage allowed during the current season
    'FTPctAllowedLastSeason B',  # Team B's free-throw percentage allowed during the previous season

    'FTPctMarginThisSeason B',  # difference between Team B's free-throw percentage and Team B's free-throw percentage allowed during the current season
    'FTPctMarginLastSeason B',  # difference between Team B's free-throw percentage and Team B's free-throw percentage allowed during the previous season

    'RebPerGameThisSeason B',  # average number of rebounds by Team B per game during the current season
    'RebPerGameLastSeason B',  # average number of rebounds by Team B per game during the previous season

    'RebAllowedPerGameThisSeason B',  # average number of rebounds allowed by Team B per game during the current season
    'RebAllowedPerGameLastSeason B',  # average number of rebounds allowed by Team B per game during the previous season

    'RebMarginPerGameThisSeason B',  # average difference between number of rebounds by Team B and number of rebounds allowed by Team B per game during the current season
    'RebMarginPerGameLastSeason B',  # average difference between number of rebounds by Team B and number of rebounds allowed by Team B per game during the previous season

    'ORebPerGameThisSeason B',  # average number of offensive rebounds by Team B per game during the current season
    'ORebPerGameLastSeason B',  # average number of offensive rebounds by Team B per game during the previous season

    'ORebAllowedPerGameThisSeason B',  # average number of offensive rebounds allowed by Team B per game during the current season
    'ORebAllowedPerGameLastSeason B',  # average number of offensive rebounds allowed by Team B per game during the previous season

    'ORebMarginPerGameThisSeason B',  # average difference between number of offensive rebounds by Team B and number of rebounds allowed by Team B per game during the current season
    'ORebMarginPerGameLastSeason B',  # average difference between number of offensive rebounds by Team B and number of rebounds allowed by Team B per game during the previous season

    'DRebPerGameThisSeason B',  # average number of defensive rebounds by Team B per game during the current season
    'DRebPerGameLastSeason B',  # average number of defensive rebounds by Team B per game during the previous season

    'DRebAllowedPerGameThisSeason B',  # average number of defensive rebounds allowed by Team B per game during the current season
    'DRebAllowedPerGameLastSeason B',  # average number of defensive rebounds allowed by Team B per game during the previous season

    'DRebMarginPerGameThisSeason B',  # average difference between number of defensive rebounds by Team B and number of rebounds allowed by Team B per game during the current season
    'DRebMarginPerGameLastSeason B',  # average difference between number of defensive rebounds by Team B and number of rebounds allowed by Team B per game during the previous season

    'AstPerGameThisSeason B',  # average number of assists by Team B per game during the current season
    'AstPerGameLastSeason B',  # average number of assists by Team B per game during the previous season

    'AstAllowedPerGameThisSeason B',  # average number of assists allowed by Team B per game during the current season
    'AstAllowedPerGameLastSeason B',  # average number of assists allowed by Team B per game during the previous season

    'AstMarginPerGameThisSeason B',  # average difference between number of assists by Team B and number of assists allowed by Team B per game during the current season
    'AstMarginPerGameLastSeason B',  # average difference between number of assists by Team B and number of assists allowed by Team B per game during the previous season

    'TOPerGameThisSeason B',  # average number of turnovers by Team B per game during the current season
    'TOPerGameLastSeason B',  # average number of turnovers by Team B per game during the previous season

    'TOAllowedPerGameThisSeason B',  # average number of turnovers forced by Team B per game during the current season
    'TOAllowedPerGameLastSeason B',  # average number of turnovers forced by Team B per game during the previous season

    'TOMarginPerGameThisSeason B',  # average difference between number of turnovers by Team B and number of turnovers forced by Team B per game during the current season
    'TOMarginPerGameLastSeason B',  # average difference between number of turnovers by Team B and number of turnovers forced by Team B per game during the previous season

    'AvgAst/TORatioThisSeason B',  # Team B's season total assists divided by Team B's season total turnovers during the current season
    'AvgAst/TORatioLastSeason B',  # Team B's season total assists divided by Team B's season total turnovers during the previous season

    'AvgAst/TORatioAllowedThisSeason B',  # Team B's season total assists allowed divided by Team B's season total turnovers forced during the current season
    'AvgAst/TORatioAllowedLastSeason B',  # Team B's season total assists allowed divided by Team B's season total turnovers forced during the previous season

    'AvgAst/TORatioMarginThisSeason B',  # (Team B's season total assists divided by Team B's season total turnovers) minus (Team B's season total assists allowed divided by Team B's season total turnovers forced) during the current season
    'AvgAst/TORatioMarginLastSeason B',  # (Team B's season total assists divided by Team B's season total turnovers) minus (Team B's season total assists allowed divided by Team B's season total turnovers forced) during the previous season

    'AvgAst/TORatio(alt)ThisSeason B',  # average of Team B's assist/turnover ratios for each game Team B has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)LastSeason B',  # average of Team B's assist/turnover ratios for each game Team B played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'AvgAst/TORatio(alt)AllowedThisSeason B',  # average of Team B's opponent's assist/turnover ratios for each game Team B has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)AllowedLastSeason B',  # average of Team B's opponent's assist/turnover ratios for each game Team B has played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'AvgAst/TORatio(alt)MarginThisSeason B',  # average of the differences between Team B's assist/turnover ratio and Team B's opponent's assist/turnover ratio for each game Team B has played during the current season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'AvgAst/TORatio(alt)MarginLastSeason B',  # average of the differences between Team B's assist/turnover ratio and Team B's opponent's assist/turnover ratio for each game Team B played during the previous season (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'StlPerGameThisSeason B',  # average number of steals by Team B per game during the current season
    'StlPerGameLastSeason B',  # average number of steals by Team B per game during the previous season

    'StlAllowedPerGameThisSeason B',  # average number of steals allowed by Team B per game during the current season
    'StlAllowedPerGameLastSeason B',  # average number of steals allowed by Team B per game during the previous season

    'StlMarginPerGameThisSeason B',  # average difference between number of steals by Team B and number of steals allowed by Team B per game during the current season
    'StlMarginPerGameLastSeason B',  # average difference between number of steals by Team B and number of steals allowed by Team B per game during the previous season

    'BlkPerGameThisSeason B',  # average number of blocks by Team B per game during the current season
    'BlkPerGameLastSeason B',  # average number of blocks by Team B per game during the previous season

    'BlkAllowedPerGameThisSeason B',  # average number of blocks allowed by Team B per game during the current season
    'BlkAllowedPerGameLastSeason B',  # average number of blocks allowed by Team B per game during the previous season

    'BlkMarginPerGameThisSeason B',  # average difference between number of blocks by Team B and number of blocks allowed by Team B per game during the current season
    'BlkMarginPerGameLastSeason B',  # average difference between number of blocks by Team B and number of blocks allowed by Team B per game during the previous season

    'PFPerGameThisSeason B',  # average number of fouls committed by Team B per game during the current season
    'PFPerGameLastSeason B',  # average number of fouls committed by Team B per game during the previous season

    'PFAllowedPerGameThisSeason B',  # average number of fouls absorbed by Team B per game during the current season
    'PFAllowedPerGameLastSeason B',  # average number of fouls absorbed by Team B per game during the previous season

    'PFMarginPerGameThisSeason B',  # average difference between number of fouls committed by Team B and number of fouls absorbed by Team B per game during the current season
    'PFMarginPerGameLastSeason B',  # average difference between number of fouls committed by Team B and number of fouls absorbed by Team B per game during the previous season

    # Team B strength of schedule

    'CurrentSoSWeightedAvgOppWinPctThenThisSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the current season
    'PastSoSWeightedAvgOppWinPctThenLastSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the
    'CurrentSoSAvgOppWinPctNowThisSeason B',  # average up-to-date win percentage (counting only the current season) of the teams which Team B has played during the current season
    'CurrentSoSAvgOppWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B has played during the current season
    'PastSoSAvgOppWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B played during the previous season

    'CurrentSoSWeightedAvgOppFlipCloseWinPctThenThisSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the current season (had the results of all of their close games been flipped)
    'PastSoSWeightedAvgOppFlipCloseWinPctThenLastSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the previous season (had the results of all of their close games been flipped)
    'CurrentSoSAvgOppFlipCloseWinPctNowThisSeason B',  # average up-to-date win percentage (counting only the current season) of the teams which Team B has played during the current season (had the results of all of their close games been flipped)
    'CurrentSoSAvgOppFlipCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B has played during the current season (had the results of all of their close games been flipped)
    'PastSoSAvgOppFlipCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B played during the previous season (had the results of all of their close games been flipped)

    'CurrentSoSWeightedAvgOppWinCloseWinPctThenThisSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the current season (had they won all of their close games)
    'PastSoSWeightedAvgOppWinCloseWinPctThenLastSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the previous season (had they won all of their close games)
    'CurrentSoSAvgOppWinCloseWinPctNowThisSeason B',  # average up-to-date win percentage (counting only the current season) of the teams which Team B has played during the current season (had they won all of their close games)
    'CurrentSoSAvgOppWinCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B has played during the current season (had they won all of their close games)
    'PastSoSAvgOppWinCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B played during the previous season (had they won all of their close games)

    'CurrentSoSWeightedAvgOppLoseCloseWinPctThenThisSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the current season (had they lost all of their close games)
    'PastSoSWeightedAvgOppLoseCloseWinPctThenLastSeason B',  # average win percentage of Team B's opponents at the time when they played Team B, weighted by number of games played by opponent going into the game against Team B, during the previous season (had they lost all of their close games)
    'CurrentSoSAvgOppLoseCloseWinPctNowThisSeason B',  # average up-to-date win percentage (counting only the current season) of the teams which Team B has played during the current season (had they lost all of their close games)
    'CurrentSoSAvgOppLoseCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B has played during the current season (had they lost all of their close games)
    'PastSoSAvgOppLoseCloseWinPctNowLastSeason B',  # average end-of-season win percentage (counting only the previous season) of the teams which Team B played during the previous season (had they lost all of their close games)

    'CurrentSoSWeightedAvgOppPointsPerGameThenThisSeason B',  # average of Team B's current-season opponents' average points scored per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average points scored per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average points scored per game during the current season
    'CurrentSoSAvgOppPointsPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average points scored per game during the previous season
    'PastSoSAvgOppPointsPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average points scored per game during the previous season

    'CurrentSoSWeightedAvgOppPointsAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average points allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average points allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average points allowed per game during the current season
    'CurrentSoSAvgOppPointsAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average points allowed per game during the previous season
    'PastSoSAvgOppPointsAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average points allowed per game during the previous season

    'CurrentSoSWeightedAvgOppPointsMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between points scored and points allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPointsMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between points scored and points allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPointsMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between points scored and points allowed per game during the current season
    'CurrentSoSAvgOppPointsMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between points scored and points allowed per game during the previous season
    'PastSoSAvgOppPointsMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between points scored and points allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGMPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of field goals made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of field goals made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of field goals made per game during the current season
    'CurrentSoSAvgOppFGMPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of field goals made per game during the previous season
    'PastSoSAvgOppFGMPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of field goals made per game during the previous season

    'CurrentSoSWeightedAvgOppFGMAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of field goals allowed per game during the current season
    'CurrentSoSAvgOppFGMAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of field goals allowed per game during the previous season
    'PastSoSAvgOppFGMAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGMMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of field goals made and number of field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGMMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of field goals made and number of field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGMMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of field goals made and number of field goals allowed per game during the current season
    'CurrentSoSAvgOppFGMMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of field goals made and number of field goals allowed per game during the previous season
    'PastSoSAvgOppFGMMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of field goals made and number of field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGAPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of field goal attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of field goal attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of field goal attempts per game during the current season
    'CurrentSoSAvgOppFGAPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of field goal attempts per game during the previous season
    'PastSoSAvgOppFGAPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of field goal attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFGAAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGAAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGAAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGAMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of field goal attempts and number of field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGAMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of field goal attempts and number of field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGAMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of field goal attempts and number of field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGAMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of field goal attempts and number of field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGAMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of field goal attempts and number of field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGPctThenThisSeason B',  # average of Team B's current-season opponents' season-long field goal percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctThenLastSeason B',  # average of Team B's previous-season opponents' season-long field goal percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long field goal percentage during the current season
    'CurrentSoSAvgOppFGPctNowLastSeason B',  # average of Team B's current-season opponents' season-long field goal percentage during the previous season
    'PastSoSAvgOppFGPctNowLastSeason B',  # average of Team B's previous-season opponents' season-long field goal percentage during the previous season

    'CurrentSoSWeightedAvgOppFGPctAllowedThenThisSeason B',  # average of Team B's current-season opponents' season-long field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctAllowedThenLastSeason B',  # average of Team B's previous-season opponents' season-long field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctAllowedNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long field goal percentage allowed during the current season
    'CurrentSoSAvgOppFGPctAllowedNowLastSeason B',  # average of Team B's current-season opponents' season-long field goal percentage allowed during the previous season
    'PastSoSAvgOppFGPctAllowedNowLastSeason B',  # average of Team B's previous-season opponents' season-long field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFGPctMarginThenThisSeason B',  # average of Team B's current-season opponents' differences between season-long field goal percentage and season-long field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGPctMarginThenLastSeason B',  # average of Team B's previous-season opponents' differences between previous-season-long field goal percentage and previous-season-long field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGPctMarginNowThisSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long field goal percentage and season-long field goal percentage allowed during the current season
    'CurrentSoSAvgOppFGPctMarginNowLastSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long field goal percentage and season-long field goal percentage allowed during the previous season
    'PastSoSAvgOppFGPctMarginNowLastSeason B',  # average of Team B's previous-season opponents' differences between season-long field goal percentage and season-long field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFGM3PerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of 3-point field goals made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3PerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of 3-point field goals made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3PerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of 3-point field goals made per game during the current season
    'CurrentSoSAvgOppFGM3PerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of 3-point field goals made per game during the previous season
    'PastSoSAvgOppFGM3PerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of 3-point field goals made per game during the previous season

    'CurrentSoSWeightedAvgOppFGM3AllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of 3-point field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3AllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of 3-point field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3AllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of 3-point field goals allowed per game during the current season
    'CurrentSoSAvgOppFGM3AllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of 3-point field goals allowed per game during the previous season
    'PastSoSAvgOppFGM3AllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of 3-point field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGM3MarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of 3-point field goals made and number of 3-point field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGM3MarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of 3-point field goals made and number of 3-point field goals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGM3MarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the current season
    'CurrentSoSAvgOppFGM3MarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the previous season
    'PastSoSAvgOppFGM3MarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of 3-point field goals made and number of 3-point field goals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3PerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of 3-point field goal attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3PerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of 3-point field goal attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3PerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of 3-point field goal attempts per game during the current season
    'CurrentSoSAvgOppFGA3PerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of 3-point field goal attempts per game during the previous season
    'PastSoSAvgOppFGA3PerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of 3-point field goal attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3AllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3AllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3AllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of 3-point field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGA3AllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of 3-point field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGA3AllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of 3-point field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFGA3MarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFGA3MarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFGA3MarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the current season
    'CurrentSoSAvgOppFGA3MarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the previous season
    'PastSoSAvgOppFGA3MarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of 3-point field goal attempts and number of 3-point field goal attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFG3PctThenThisSeason B',  # average of Team B's current-season opponents' season-long 3-point field goal percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctThenLastSeason B',  # average of Team B's previous-season opponents' season-long 3-point field goal percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long 3-point field goal percentage during the current season
    'CurrentSoSAvgOppFG3PctNowLastSeason B',  # average of Team B's current-season opponents' season-long 3-point field goal percentage during the previous season
    'PastSoSAvgOppFG3PctNowLastSeason B',  # average of Team B's previous-season opponents' season-long 3-point field goal percentage during the previous season

    'CurrentSoSWeightedAvgOppFG3PctAllowedThenThisSeason B',  # average of Team B's current-season opponents' season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctAllowedThenLastSeason B',  # average of Team B's previous-season opponents' season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctAllowedNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long 3-point field goal percentage allowed during the current season
    'CurrentSoSAvgOppFG3PctAllowedNowLastSeason B',  # average of Team B's current-season opponents' season-long 3-point field goal percentage allowed during the previous season
    'PastSoSAvgOppFG3PctAllowedNowLastSeason B',  # average of Team B's previous-season opponents' season-long 3-point field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFG3PctMarginThenThisSeason B',  # average of Team B's current-season opponents' differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFG3PctMarginThenLastSeason B',  # average of Team B's previous-season opponents' differences between previous-season-long 3-point field goal percentage and previous-season-long 3-point field goal percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFG3PctMarginNowThisSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the current season
    'CurrentSoSAvgOppFG3PctMarginNowLastSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the previous season
    'PastSoSAvgOppFG3PctMarginNowLastSeason B',  # average of Team B's previous-season opponents' differences between season-long 3-point field goal percentage and season-long 3-point field goal percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFTMPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of free-throws made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of free-throws made per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of free-throws made per game during the current season
    'CurrentSoSAvgOppFTMPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of free-throws made per game during the previous season
    'PastSoSAvgOppFTMPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of free-throws made per game during the previous season

    'CurrentSoSWeightedAvgOppFTMAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of successful free-throws allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of successful free-throws allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of successful free-throws allowed per game during the current season
    'CurrentSoSAvgOppFTMAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of successful free-throws allowed per game during the previous season
    'PastSoSAvgOppFTMAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of successful free-throws allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTMMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of free-throws made and number of successful free-throws allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTMMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of free-throws made and number of successful free-throws allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTMMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of free-throws made and number of successful free-throws allowed per game during the current season
    'CurrentSoSAvgOppFTMMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of free-throws made and number of successful free-throws allowed per game during the previous season
    'PastSoSAvgOppFTMMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of free-throws made and number of successful free-throws allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTAPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of free-throw attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of free-throw attempts per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of free-throw attempts per game during the current season
    'CurrentSoSAvgOppFTAPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of free-throw attempts per game during the previous season
    'PastSoSAvgOppFTAPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of free-throw attempts per game during the previous season

    'CurrentSoSWeightedAvgOppFTAAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average number of free-throw attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average number of free-throw attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average number of free-throw attempts allowed per game during the current season
    'CurrentSoSAvgOppFTAAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average number of free-throw attempts allowed per game during the previous season
    'PastSoSAvgOppFTAAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average number of free-throw attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTAMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between number of free-throw attempts and number of free-throw attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTAMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between number of free-throw attempts and number of free-throw attempts allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTAMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the current season
    'CurrentSoSAvgOppFTAMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the previous season
    'PastSoSAvgOppFTAMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between number of free-throw attempts and number of free-throw attempts allowed per game during the previous season

    'CurrentSoSWeightedAvgOppFTPctThenThisSeason B',  # average of Team B's current-season opponents' season-long free-throw percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctThenLastSeason B',  # average of Team B's previous-season opponents' season-long free-throw percentage, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long free-throw percentage during the current season
    'CurrentSoSAvgOppFTPctNowLastSeason B',  # average of Team B's current-season opponents' season-long free-throw percentage during the previous season
    'PastSoSAvgOppFTPctNowLastSeason B',  # average of Team B's previous-season opponents' season-long free-throw percentage during the previous season

    'CurrentSoSWeightedAvgOppFTPctAllowedThenThisSeason B',  # average of Team B's current-season opponents' season-long free-throw percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctAllowedThenLastSeason B',  # average of Team B's previous-season opponents' season-long free-throw percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctAllowedNowThisSeason B',  # average of Team B's current-season opponents' up-to-date season-long free-throw percentage allowed during the current season
    'CurrentSoSAvgOppFTPctAllowedNowLastSeason B',  # average of Team B's current-season opponents' season-long free-throw percentage allowed during the previous season
    'PastSoSAvgOppFTPctAllowedNowLastSeason B',  # average of Team B's previous-season opponents' season-long free-throw percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppFTPctMarginThenThisSeason B',  # average of Team B's current-season opponents' differences between season-long free-throw percentage and season-long free-throw percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppFTPctMarginThenLastSeason B',  # average of Team B's previous-season opponents' differences between previous-season-long free-throw percentage and previous-season-long free-throw percentage allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppFTPctMarginNowThisSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long free-throw percentage and season-long free-throw percentage allowed during the current season
    'CurrentSoSAvgOppFTPctMarginNowLastSeason B',  # average of Team B's current-season opponents' up-to-date differences between season-long free-throw percentage and season-long free-throw percentage allowed during the previous season
    'PastSoSAvgOppFTPctMarginNowLastSeason B',  # average of Team B's previous-season opponents' differences between season-long free-throw percentage and season-long free-throw percentage allowed during the previous season

    'CurrentSoSWeightedAvgOppRebPerGameThenThisSeason B',  # average of Team B's current-season opponents' average rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average rebounds per game during the current season
    'CurrentSoSAvgOppRebPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average rebounds per game during the previous season
    'PastSoSAvgOppRebPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppRebAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average rebounds allowed per game during the current season
    'CurrentSoSAvgOppRebAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average rebounds allowed per game during the previous season
    'PastSoSAvgOppRebAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppRebMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between rebounds and rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppRebMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between rebounds and rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppRebMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between rebounds and rebounds allowed per game during the current season
    'CurrentSoSAvgOppRebMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between rebounds and rebounds allowed per game during the previous season
    'PastSoSAvgOppRebMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between rebounds and rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppORebPerGameThenThisSeason B',  # average of Team B's current-season opponents' average offensive rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average offensive rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average offensive rebounds per game during the current season
    'CurrentSoSAvgOppORebPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average offensive rebounds per game during the previous season
    'PastSoSAvgOppORebPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average offensive rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppORebAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average offensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average offensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average offensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppORebAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average offensive rebounds allowed per game during the previous season
    'PastSoSAvgOppORebAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average offensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppORebMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between offensive rebounds and offensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppORebMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between offensive rebounds and offensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppORebMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between offensive rebounds and offensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppORebMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between offensive rebounds and offensive rebounds allowed per game during the previous season
    'PastSoSAvgOppORebMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between offensive rebounds and offensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppDRebPerGameThenThisSeason B',  # average of Team B's current-season opponents' average defensive rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average defensive rebounds per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average defensive rebounds per game during the current season
    'CurrentSoSAvgOppDRebPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average defensive rebounds per game during the previous season
    'PastSoSAvgOppDRebPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average defensive rebounds per game during the previous season

    'CurrentSoSWeightedAvgOppDRebAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average defensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average defensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average defensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppDRebAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average defensive rebounds allowed per game during the previous season
    'PastSoSAvgOppDRebAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average defensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppDRebMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between defensive rebounds and defensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppDRebMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between defensive rebounds and defensive rebounds allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppDRebMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between defensive rebounds and defensive rebounds allowed per game during the current season
    'CurrentSoSAvgOppDRebMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between defensive rebounds and defensive rebounds allowed per game during the previous season
    'PastSoSAvgOppDRebMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between defensive rebounds and defensive rebounds allowed per game during the previous season

    'CurrentSoSWeightedAvgOppAstPerGameThenThisSeason B',  # average of Team B's current-season opponents' average assists per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average assists per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average assists per game during the current season
    'CurrentSoSAvgOppAstPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average assists per game during the previous season
    'PastSoSAvgOppAstPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average assists per game during the previous season

    'CurrentSoSWeightedAvgOppAstAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average assists allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average assists allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average assists allowed per game during the current season
    'CurrentSoSAvgOppAstAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average assists allowed per game during the previous season
    'PastSoSAvgOppAstAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average assists allowed per game during the previous season

    'CurrentSoSWeightedAvgOppAstMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between assists and assists allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAstMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between assists and assists allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAstMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between assists and assists allowed per game during the current season
    'CurrentSoSAvgOppAstMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between assists and assists allowed per game during the previous season
    'PastSoSAvgOppAstMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between assists and assists allowed per game during the previous season

    'CurrentSoSWeightedAvgOppTOPerGameThenThisSeason B',  # average of Team B's current-season opponents' average turnovers per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average turnovers per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average turnovers per game during the current season
    'CurrentSoSAvgOppTOPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average turnovers per game during the previous season
    'PastSoSAvgOppTOPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average turnovers per game during the previous season

    'CurrentSoSWeightedAvgOppTOAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average turnovers forced per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average turnovers forced per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average turnovers forced per game during the current season
    'CurrentSoSAvgOppTOAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average turnovers forced per game during the previous season
    'PastSoSAvgOppTOAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average turnovers forced per game during the previous season

    'CurrentSoSWeightedAvgOppTOMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between turnovers and turnovers forced per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppTOMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between turnovers and turnovers forced per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppTOMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between turnovers and turnovers forced per game during the current season
    'CurrentSoSAvgOppTOMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between turnovers and turnovers forced per game during the previous season
    'PastSoSAvgOppTOMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between turnovers and turnovers forced per game during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioThenThisSeason B',  # average of Team B's current-season opponents' current-season-long assist/turnover ratios (total assists / total turnovers), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long assist/turnover ratios (total assists / total turnovers), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioNowThisSeason B',  # average of Team B's current-season opponents' up-to-date assist/turnover ratios (total assists / total turnovers) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioNowLastSeason B',  # average of Team B's current-season opponents' season-long assist/turnover ratios (total assists / total turnovers) during the previous season
    'PastSoSAvgOppAvgAst/TORatioNowLastSeason B',  # average of Team B's previous-season opponents' season-long assist/turnover ratios (total assists / total turnovers) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioAllowedThenThisSeason B',  # average of Team B's current-season opponents' current-season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioAllowedThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioAllowedNowThisSeason B',  # average of Team B's current-season opponents' up-to-date assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioAllowedNowLastSeason B',  # average of Team B's current-season opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the previous season
    'PastSoSAvgOppAvgAst/TORatioAllowedNowLastSeason B',  # average of Team B's previous-season opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatioMarginThenThisSeason B',  # average of Team B's current-season opponents' current-season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppAvgAst/TORatioMarginThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced), recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppAvgAst/TORatioMarginNowThisSeason B',  # average of Team B's current-season opponents' up-to-date differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the current season
    'CurrentSoSAvgOppAvgAst/TORatioMarginNowLastSeason B',  # average of Team B's current-season opponents' season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the previous season
    'PastSoSAvgOppAvgAst/TORatioMarginNowLastSeason B',  # average of Team B's previous-season opponents' season-long differences between assist/turnover ratio (total assists / total turnovers) and assist/turnover ratio allowed (total assists allowed / total turnovers forced) during the previous season

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)ThenThisSeason B',  # average of Team B's current-season opponents' current-season-long assist/turnover ratios, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)ThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long assist/turnover ratios, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)NowThisSeason B',  # average of Team B's current-season opponents' up-to-date assist/turnover ratios during the current season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)NowLastSeason B',  # average of Team B's current-season opponents' season-long assist/turnover ratios during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSAvgOppAvgAst/TORatio(alt)NowLastSeason B',  # average of Team B's previous-season opponents' season-long assist/turnover ratios during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenThisSeason B',  # average of Team B's current-season opponents' current-season-long assist/turnover ratios allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long assist/turnover ratios allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowThisSeason B',  # average of Team B's current-season opponents' up-to-date assist/turnover ratios allowed during the current season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason B',  # average of Team B's current-season opponents' season-long assist/turnover ratios allowed during the previous season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)
    'PastSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason B',  # average of Team B's previous-season opponents' season-long assist/turnover ratios allowed during the previous season (ratios calculated by averaging the assist/turnover ratios allowed from each game, not by dividing total assists allowed / total turnovers forced)

    'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenThisSeason B',  # average of Team B's current-season opponents' current-season-long differences between assist/turnover ratio and assist/turnover ratio allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenLastSeason B',  # average of Team B's previous-season opponents' previous-season-long differences between assist/turnover ratio and assist/turnover ratio allowed, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowThisSeason B',  # average of Team B's current-season opponents' up-to-date differences between assist/turnover ratio and assist/turnover ratio allowed during the current season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason B',  # average of Team B's current-season opponents' season-long differences between assist/turnover ratio and assist/turnover ratio allowed during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
    'PastSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason B',  # average of Team B's previous-season opponents' season-long differences between assist/turnover ratio and assist/turnover ratio allowed during the previous season (ratios calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)

    'CurrentSoSWeightedAvgOppStlPerGameThenThisSeason B',  # average of Team B's current-season opponents' average steals per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average steals per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average steals per game during the current season
    'CurrentSoSAvgOppStlPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average steals per game during the previous season
    'PastSoSAvgOppStlPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average steals per game during the previous season

    'CurrentSoSWeightedAvgOppStlAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average steals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average steals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average steals allowed per game during the current season
    'CurrentSoSAvgOppStlAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average steals allowed per game during the previous season
    'PastSoSAvgOppStlAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average steals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppStlMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between steals and steals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppStlMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between steals and steals allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppStlMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between steals and steals allowed per game during the current season
    'CurrentSoSAvgOppStlMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between steals and steals allowed per game during the previous season
    'PastSoSAvgOppStlMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between steals and steals allowed per game during the previous season

    'CurrentSoSWeightedAvgOppBlkPerGameThenThisSeason B',  # average of Team B's current-season opponents' average blocks per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average blocks per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average blocks per game during the current season
    'CurrentSoSAvgOppBlkPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average blocks per game during the previous season
    'PastSoSAvgOppBlkPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average blocks per game during the previous season

    'CurrentSoSWeightedAvgOppBlkAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average blocks allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average blocks allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average blocks allowed per game during the current season
    'CurrentSoSAvgOppBlkAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average blocks allowed per game during the previous season
    'PastSoSAvgOppBlkAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average blocks allowed per game during the previous season

    'CurrentSoSWeightedAvgOppBlkMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between blocks and blocks allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppBlkMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between blocks and blocks allowed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppBlkMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between blocks and blocks allowed per game during the current season
    'CurrentSoSAvgOppBlkMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between blocks and blocks allowed per game during the previous season
    'PastSoSAvgOppBlkMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between blocks and blocks allowed per game during the previous season

    'CurrentSoSWeightedAvgOppPFPerGameThenThisSeason B',  # average of Team B's current-season opponents' average fouls committed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average fouls committed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average fouls committed per game during the current season
    'CurrentSoSAvgOppPFPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average fouls committed per game during the previous season
    'PastSoSAvgOppPFPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average fouls committed per game during the previous season

    'CurrentSoSWeightedAvgOppPFAllowedPerGameThenThisSeason B',  # average of Team B's current-season opponents' average fouls absorbed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFAllowedPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average fouls absorbed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFAllowedPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average fouls absorbed per game during the current season
    'CurrentSoSAvgOppPFAllowedPerGameNowLastSeason B',  # average of Team B's current-season opponents' season average fouls absorbed per game during the previous season
    'PastSoSAvgOppPFAllowedPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average fouls absorbed per game during the previous season

    'CurrentSoSWeightedAvgOppPFMarginPerGameThenThisSeason B',  # average of Team B's current-season opponents' average difference between fouls committed and fouls absorbed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'PastSoSWeightedAvgOppPFMarginPerGameThenLastSeason B',  # average of Team B's previous-season opponents' average difference between fouls committed and fouls absorbed per game, recorded immediately prior to playing Team B, and weighted by number of games each opponent had played at that time
    'CurrentSoSAvgOppPFMarginPerGameNowThisSeason B',  # average of Team B's current-season opponents' up-to-date average difference between fouls committed and fouls absorbed per game during the current season
    'CurrentSoSAvgOppPFMarginPerGameNowLastSeason B',  # average of Team B's current-season opponents' season-long average difference between fouls committed and fouls absorbed per game during the previous season
    'PastSoSAvgOppPFMarginPerGameNowLastSeason B',  # average of Team B's previous-season opponents' season average difference between fouls committed and fouls absorbed per game during the previous season
])

inputData = emptyInputDF.copy()
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# read in raw game data
rawGamesDF = pd.read_csv(rootDirectory + '/Data/Regular_Season_Detailed_Results.csv')

# convert from legacy Massey IDs to Massey URLs
massey_conversion_df = pd.read_csv(rootDirectory + '/Data/Team_Name_Conversion.csv')
massey_id_to_massey_url_dict = dict(zip(massey_conversion_df.dropna(subset=['Massey ID (Legacy)'])['Massey ID (Legacy)'], massey_conversion_df.dropna(subset=['Massey ID (Legacy)'])['Massey URL']))
rawGamesDF['WTeamID'] = rawGamesDF['WTeamID'].map(massey_id_to_massey_url_dict)
rawGamesDF['LTeamID'] = rawGamesDF['LTeamID'].map(massey_id_to_massey_url_dict)

# identify close games
rawGamesDF['Close'] = np.logical_or(rawGamesDF.loc[:, 'NumOT'], rawGamesDF.loc[:, 'WScore'] - rawGamesDF.loc[:, 'LScore'] <= 3)

# to prevent division by zero, set a small positive minimum for field goal attempts, 3-point field goal attempts, free-throw attempts, and turnovers
columns_to_convert = ['WFGA', 'LFGA', 'WFGA3', 'LFGA3', 'WFTA', 'LFTA', 'WTO', 'LTO']
rawGamesDF[columns_to_convert] = rawGamesDF[columns_to_convert].astype(float)
rawGamesDF.loc[:, 'WFGA'] = rawGamesDF.loc[:, 'WFGA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFGA'] = rawGamesDF.loc[:, 'LFGA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WFGA3'] = rawGamesDF.loc[:, 'WFGA3'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFGA3'] = rawGamesDF.loc[:, 'LFGA3'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WFTA'] = rawGamesDF.loc[:, 'WFTA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LFTA'] = rawGamesDF.loc[:, 'LFTA'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'WTO'] = rawGamesDF.loc[:, 'WTO'].map(lambda x: max(x, 2 ** -64))
rawGamesDF.loc[:, 'LTO'] = rawGamesDF.loc[:, 'LTO'].map(lambda x: max(x, 2 ** -64))

# add stop-sign row at the end of rawGamesDF
rawGamesDF.loc[len(rawGamesDF), 'Season'] = 9999

# read in set of all teams
teams = pd.read_csv(rootDirectory + '/Data/Team_Name_Conversion.csv').iloc[:, 0:2]
teams.set_index('Massey URL', inplace=True)

# initialize weights for strength of schedule calculations (increase from 0 to an upper limit of 1)
GamesPlayedWeights = .00390625 * (((np.array(list(range(45)))) * (np.array(list(range(45)))) * (np.array(list(range(45)))) * (np.array(list(range(45))))) / (((.0625 * (np.array(list(range(45)))) * (np.array(list(range(45))))) + 1) * ((.0625 * (np.array(list(range(45)))) * (np.array(list(range(45))))) + 1)))  # used to weight the importance of opponents' records by the number of games they've played

# define dictionary of numerical values for home-court advantage
HomeAdvDict = {'H': 1, 'N': 0, 'A': -1}

# initialize dictionary to store refined data for each season, read in pre-refined data from the first season
seasonTotalsDict = {startingSeason: pd.read_csv(rootDirectory + '/Data/Team_Stats_Season_Totals' + '/Team_Stats_Season_Totals_' + str(startingSeason) + '.csv')}

# set indexes of season total dataframes to the teams' four-digit identification numbers
for season in seasonTotalsDict:
    seasonTotalsDict[season].set_index('Team ID', inplace=True)

# set starting season and day before first season and day
season = 1891  # set to arbitrary year prior to 2003 (start of data)
dayNum = -1
upToMinuteSeasonTotals = pd.DataFrame({})
TeamAWins = True
startTime = datetime.now()

# iterate through each game in rawGamesDF, starting from first season after startingSeason
for index in range((rawGamesDF['Season'] == startingSeason + 1).idxmax(), len(rawGamesDF)):

    # if this is the first game of a new season
    if rawGamesDF.iloc[index, 0] != season:

        # if at least 1 season's worth of games have been processed
        if season > 1891:
            # make final up-to-minute calculations for previous season
            upToMinuteSeasonTotals.loc[:, 'WinPct'] = upToMinuteSeasonTotals.loc[:, 'Wins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FlipCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'FlipCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'WinCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'WinCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'LoseCloseWinPct'] = upToMinuteSeasonTotals.loc[:, 'LoseCloseWins'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsPerGame'] = upToMinuteSeasonTotals.loc[:, 'Points'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'PointsAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PointsMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'PointsPerGame'] - upToMinuteSeasonTotals.loc[:, 'PointsAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGMPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGMAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGMAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGMMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGMPerGame'] - upToMinuteSeasonTotals.loc[:, 'FGMAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGAPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGAAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGAAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGAMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGAPerGame'] - upToMinuteSeasonTotals.loc[:, 'FGAAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGPctMargin'] = upToMinuteSeasonTotals.loc[:, 'FGPct'] - upToMinuteSeasonTotals.loc[:, 'FGPctAllowed']
            upToMinuteSeasonTotals.loc[:, 'FGM3PerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGM3AllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3Allowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGM3MarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGM3PerGame'] - upToMinuteSeasonTotals.loc[:, 'FGM3AllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FGA3PerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGA3AllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3Allowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FGA3MarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FGA3PerGame'] - upToMinuteSeasonTotals.loc[:, 'FGA3AllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FG3PctMargin'] = upToMinuteSeasonTotals.loc[:, 'FG3Pct'] - upToMinuteSeasonTotals.loc[:, 'FG3PctAllowed']
            upToMinuteSeasonTotals.loc[:, 'FTMPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTM'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTMAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTMAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTMMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTMPerGame'] - upToMinuteSeasonTotals.loc[:, 'FTMAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FTAPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTA'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTAAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTAAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'FTAMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'FTAPerGame'] - upToMinuteSeasonTotals.loc[:, 'FTAAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'FTPctMargin'] = upToMinuteSeasonTotals.loc[:, 'FTPct'] - upToMinuteSeasonTotals.loc[:, 'FTPctAllowed']
            upToMinuteSeasonTotals.loc[:, 'RebPerGame'] = (upToMinuteSeasonTotals.loc[:, 'OReb'] + upToMinuteSeasonTotals.loc[:, 'DReb']) / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'RebAllowedPerGame'] = (upToMinuteSeasonTotals.loc[:, 'ORebAllowed'] + upToMinuteSeasonTotals.loc[:, 'DRebAllowed']) / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'RebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'RebPerGame'] - upToMinuteSeasonTotals.loc[:, 'RebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'ORebPerGame'] = upToMinuteSeasonTotals.loc[:, 'OReb'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'ORebAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'ORebAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'ORebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'ORebPerGame'] - upToMinuteSeasonTotals.loc[:, 'ORebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'DRebPerGame'] = upToMinuteSeasonTotals.loc[:, 'DReb'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'DRebAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'DRebAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'DRebMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'DRebPerGame'] - upToMinuteSeasonTotals.loc[:, 'DRebAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'AstPerGame'] = upToMinuteSeasonTotals.loc[:, 'Ast'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'AstAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'AstAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'AstMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'AstPerGame'] - upToMinuteSeasonTotals.loc[:, 'AstAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'TOPerGame'] = upToMinuteSeasonTotals.loc[:, 'TO'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'TOAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'TOAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'TOMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'TOPerGame'] - upToMinuteSeasonTotals.loc[:, 'TOAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatioMargin'] = upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio'] - upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatioAllowed']
            upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)Margin'] = upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)'] - upToMinuteSeasonTotals.loc[:, 'AvgAst/TORatio(alt)Allowed']
            upToMinuteSeasonTotals.loc[:, 'StlPerGame'] = upToMinuteSeasonTotals.loc[:, 'Stl'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'StlAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'StlAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'StlMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'StlPerGame'] - upToMinuteSeasonTotals.loc[:, 'StlAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'BlkPerGame'] = upToMinuteSeasonTotals.loc[:, 'Blk'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'BlkAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'BlkAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'BlkMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'BlkPerGame'] - upToMinuteSeasonTotals.loc[:, 'BlkAllowedPerGame']
            upToMinuteSeasonTotals.loc[:, 'PFPerGame'] = upToMinuteSeasonTotals.loc[:, 'PF'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PFAllowedPerGame'] = upToMinuteSeasonTotals.loc[:, 'PFAllowed'] / upToMinuteSeasonTotals.loc[:, 'GamesPlayed']
            upToMinuteSeasonTotals.loc[:, 'PFMarginPerGame'] = upToMinuteSeasonTotals.loc[:, 'PFPerGame'] - upToMinuteSeasonTotals.loc[:, 'PFAllowedPerGame']
            # strength of schedule
            for teamID in upToMinuteSeasonTotals.index:
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'WinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFlipCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FlipCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppWinCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'WinCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppLoseCloseWinPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'LoseCloseWinPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPointsMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PointsMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGMMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGMMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGAMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGAMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGPctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGPctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3PerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3PerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3AllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3AllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGM3MarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGM3MarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3PerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3PerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3AllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3AllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFGA3MarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FGA3MarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3Pct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3PctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFG3PctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FG3PctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTMMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTMMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTAMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTAMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPct'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPctAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppFTPctMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'FTPctMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppRebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'RebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppORebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'ORebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppDRebMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'DRebMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAstMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AstMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppTOMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'TOMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioAllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatioMarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatioMargin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)Now'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)AllowedNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppAvgAst/TORatio(alt)MarginNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Margin'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppStlMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'StlMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppBlkMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'BlkMarginPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFAllowedPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFAllowedPerGame'].mean()
                upToMinuteSeasonTotals.loc[teamID, 'SoSAvgOppPFMarginPerGameNow'] = upToMinuteSeasonTotals.loc[upToMinuteSeasonTotals.loc[teamID, 'ListOpponentsTeamID'], 'PFMarginPerGame'].mean()

            # memoize end-of-season totals
            seasonTotalsDict[season] = upToMinuteSeasonTotals.copy()
            # write previous season totals
            seasonTotalsDict[int(season)].to_csv(rootDirectory + '/Data/Season_Totals' + 'Season_Totals_' + str(int(season)) + '.csv', index=False)
            print('SeasonTotals_' + str(int(season)) + '.csv written')
            # write input data
            inputData.to_csv(rootDirectory + '/Data/Training_Data' + '/Training_Data_' + str(int(season)) + '.csv', index=False)
            print('Training_Data_' + str(int(season)) + '.csv written')
        # advance to next season
        season = rawGamesDF.iloc[index, 0]
        if season == 9999:
            break
        print('Season:', int(season), '   Current time:', str(datetime.now())[:-7])
        # reset day number
        dayNum = -1
        # reset input dataframe
        inputData = emptyInputDF.copy()
        # reset season totals
        upToMinuteSeasonTotals = pd.DataFrame(columns=['Team ID',  # 4-digit numerical team identification number
                                                       # season totals
                                                       'GamesPlayed',  # total games played during the current season
                                                       'Wins',  # total wins during the current season
                                                       'CloseGames',  # total close games played during the current season
                                                       'FlipCloseWins',  # total wins during the current season (had the results of all of their close games been flipped)
                                                       'WinCloseWins',  # total wins during the current season (had they won all of their close games)
                                                       'LoseCloseWins',  # total wins during the current season (had they lost all of their close games)
                                                       'Points',  # total points scored during the current season
                                                       'PointsAllowed',  # total points allowed during the current season
                                                       'FGM',  # total field goals made during the current season
                                                       'FGMAllowed',  # total field goals allowed during the current season
                                                       'FGA',  # total field goal attempts during the current season
                                                       'FGAAllowed',  # total field goal attempts allowed during the current season
                                                       'FGPct',  # total field goals made divided by total field goal attempts during the current season
                                                       'FGPctAllowed',  # total field goals allowed divided by total field goal attempts allowed during the current season
                                                       'FGM3',  # total 3-point field goals made during the current season
                                                       'FGM3Allowed',  # total 3-point field goals allowed during the current season
                                                       'FGA3',  # total 3 point field goal attempts during the current season
                                                       'FGA3Allowed',  # total 3-point field goal attempts allowed during the current season
                                                       'FG3Pct',  # total 3-point field goals made divided by total 3-point field goal attempts during the current season
                                                       'FG3PctAllowed',  # total 3-point field goals allowed divided by total 3-point field goal attempts allowed during the current season
                                                       'FTM',  # total free-throws made during the current season
                                                       'FTMAllowed',  # total successful free-throws allowed during the current season
                                                       'FTA',  # total free-throw attempts during the current season
                                                       'FTAAllowed',  # total free-throw attempts allowed during the current season
                                                       'FTPct',  # total free-throws made divided by total free-throw attempts during the current season
                                                       'FTPctAllowed',  # total successful free-throws allowed divided by total free-throw attempts allowed during the current season
                                                       'OReb',  # total offensive rebounds during the current season
                                                       'ORebAllowed',  # total offensive rebounds allowed during the current season
                                                       'DReb',  # total defensive rebounds during the current season
                                                       'DRebAllowed',  # total defensive rebounds allowed during the current season
                                                       'Ast',  # total assists during the current season
                                                       'AstAllowed',  # total assists allowed during the current season
                                                       'TO',  # total turnovers during the current season
                                                       'TOAllowed',  # total turnovers forced during the current season
                                                       'AvgAst/TORatio',  # total assists divided by total turnovers during the current season
                                                       'AvgAst/TORatioAllowed',  # total assists allowed divided by total turnovers forced during the current season
                                                       'AvgAst/TORatio(alt)',  # average of assist/turnover ratios during the current season (calculated by averaging each game's assist/turnover ratio, not by dividing total assists / total turnovers)
                                                       'AvgAst/TORatio(alt)Allowed',  # average of opposing assist/turnover ratios during the current season (calculated by averaging each game's assist/turnover ratio, not by dividing total assists / total turnovers)
                                                       'Stl',  # total steals during the current season
                                                       'StlAllowed',  # total steals allowed during the current season
                                                       'Blk',  # total blocks during the current season
                                                       'BlkAllowed',  # total blocks allowed during the current season
                                                       'PF',  # total fouls committed during the current season
                                                       'PFAllowed',  # total fouls absorbed during the current season
                                                       # strength of schedule
                                                       'ListOpponentsTeamID',  # list of opponents' 4-digit team identification numbers
                                                       'SoSSumOppGamesPlayedWeightsDenominator',  # sum of weights applied to opponents' stats based on number of games they'd played prior to their contest against this team
                                                       'SoSWeightedAvgOpponentsWinPct',  # weighted average of opponents' win percentages entering contest against this team
                                                       'SoSWeightedAvgOpponentsFlipCloseWinPct',  # weighted average of opponents' win percentages (had the results of all of their close games been flipped) entering contest against this team
                                                       'SoSWeightedAvgOpponentsWinCloseWinPct',  # weighted average of opponents' win percentages (had they won all of their close games) entering contest against this team
                                                       'SoSWeightedAvgOpponentsLoseCloseWinPct',  # weighted average of opponents' win percentages (had they lost all of their close games) entering contest against this team
                                                       'SoSWeightedAvgOpponentsPointsPerGame',  # weighted average of opponents' average points per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPointsAllowedPerGame',  # weighted average of opponents' average points per game allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGMPerGame',  # weighted average of opponents' average field goals made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGMAllowedPerGame',  # weighted average of opponents' average field goals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGAPerGame',  # weighted average of opponents' average field goal attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGAAllowedPerGame',  # weighted average of opponents' average field goal attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGPct',  # weighted average of opponents' season-long field goal percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGPctAllowed',  # weighted average of opponents' season-long field goal percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGM3PerGame',  # weighted average of opponents' average 3-point field goals made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGM3AllowedPerGame',  # weighted average of opponents' average 3-point field goals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGA3PerGame',  # weighted average of opponents' average 3-point field goal attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFGA3AllowedPerGame',  # weighted average of opponents' average 3-point field goal attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFG3Pct',  # weighted average of opponents' season-long 3-point field goal percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFG3PctAllowed',  # weighted average of opponents' season-long 3-point field goal percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTMPerGame',  # weighted average of opponents' average free throws made per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTMAllowedPerGame',  # weighted average of opponents' average successful free throws allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTAPerGame',  # weighted average of opponents' average free throw attempts per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTAAllowedPerGame',  # weighted average of opponents' average free throw attempts allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTPct',  # weighted average of opponents' season-long free-throw percentage entering contest against this team
                                                       'SoSWeightedAvgOpponentsFTPctAllowed',  # weighted average of opponents' season-long free-throw percentage allowed entering contest against this team
                                                       'SoSWeightedAvgOpponentsORebPerGame',  # weighted average of opponents' average offensive rebounds per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsORebAllowedPerGame',  # weighted average of opponents' average offensive rebounds allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsDRebPerGame',  # weighted average of opponents' average defensive rebounds per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsDRebAllowedPerGame',  # weighted average of opponents' average defensive rebounds allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAstPerGame',  # weighted average of opponents' average assists per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAstAllowedPerGame',  # weighted average of opponents' average assists allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsTOPerGame',  # weighted average of opponents' average turnovers per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsTOAllowedPerGame',  # weighted average of opponents' average turnovers forced per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsAst/TORatio',  # weighted average of opponents' season-long assist/turnover ratios (total assists / total turnovers) entering contest against this team
                                                       'SoSWeightedAvgOpponentsAst/TORatioAllowed',  # weighted average of opponents' season-long assist/turnover ratios allowed (total assists allowed / total turnovers forced) entering contest against this team (calculated by averaging each team's total assists allowed divided by that team's total turnovers forced)
                                                       'SoSWeightedAvgOpponentsAst/TORatio(alt)',  # weighted average of opponents' average of assist/turnover ratios entering contest against this team (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
                                                       'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed',  # weighted average of opponents' average of assist/turnover ratios allowed entering contest against this team (calculated by averaging the assist/turnover ratios from each game, not by dividing total assists / total turnovers)
                                                       'SoSWeightedAvgOpponentsStlPerGame',  # weighted average of opponents' average steals per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsStlAllowedPerGame',  # weighted average of opponents' average steals allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsBlkPerGame',  # weighted average of opponents' average blocks per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsBlkAllowedPerGame',  # weighted average of opponents' average blocks allowed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPFPerGame',  # weighted average of opponents' average fouls committed per game entering contest against this team
                                                       'SoSWeightedAvgOpponentsPFAllowedPerGame'])  # weighted average of opponents' average fouls absorbed per game entering contest against this team
        upToMinuteSeasonTotals.set_index('Team ID', inplace=True)
        for team in teams.index:
            upToMinuteSeasonTotals.loc[team] = {
                'GamesPlayed': 0, 'Wins': 0.0, 'CloseGames': 0.0, 'FlipCloseWins': 0.0, 
                'WinCloseWins': 0.0, 'LoseCloseWins': 0.0, 'Points': 0.0, 'PointsAllowed': 0.0, 
                'FGM': 0.0, 'FGMAllowed': 0.0, 'FGA': 0.0, 'FGAAllowed': 0.0, 'FGPct': 0.0, 
                'FGPctAllowed': 0.0, 'FGM3': 0.0, 'FGM3Allowed': 0.0, 'FGA3': 0.0, 
                'FGA3Allowed': 0.0, 'FG3Pct': 0.0, 'FG3PctAllowed': 0.0, 'FTM': 0.0, 
                'FTMAllowed': 0.0, 'FTA': 0.0, 'FTAAllowed': 0.0, 'FTPct': 0.0, 
                'FTPctAllowed': 0.0, 'OReb': 0.0, 'ORebAllowed': 0.0, 'DReb': 0.0, 
                'DRebAllowed': 0.0, 'Ast': 0.0, 'AstAllowed': 0.0, 'TO': 0.0, 'TOAllowed': 0.0, 
                'AvgAst/TORatio': 0.0, 'AvgAst/TORatioAllowed': 0.0, 'AvgAst/TORatio(alt)': 0.0, 
                'AvgAst/TORatio(alt)Allowed': 0.0, 'Stl': 0.0, 'StlAllowed': 0.0, 'Blk': 0.0, 
                'BlkAllowed': 0.0, 'PF': 0.0, 'PFAllowed': 0.0, 'ListOpponentsTeamID': [], 
                'SoSSumOppGamesPlayedWeightsDenominator': 0.0, 'SoSWeightedAvgOpponentsWinPct': 0.0, 
                'SoSWeightedAvgOpponentsFlipCloseWinPct': 0.0, 'SoSWeightedAvgOpponentsWinCloseWinPct': 0.0, 
                'SoSWeightedAvgOpponentsLoseCloseWinPct': 0.0, 'SoSWeightedAvgOpponentsPointsPerGame': 0.0, 
                'SoSWeightedAvgOpponentsPointsAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGMPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGMAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGAPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGAAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGPct': 0.0, 
                'SoSWeightedAvgOpponentsFGPctAllowed': 0.0, 'SoSWeightedAvgOpponentsFGM3PerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGM3AllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFGA3PerGame': 0.0, 
                'SoSWeightedAvgOpponentsFGA3AllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFG3Pct': 0.0, 
                'SoSWeightedAvgOpponentsFG3PctAllowed': 0.0, 'SoSWeightedAvgOpponentsFTMPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFTMAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFTAPerGame': 0.0, 
                'SoSWeightedAvgOpponentsFTAAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsFTPct': 0.0, 
                'SoSWeightedAvgOpponentsFTPctAllowed': 0.0, 'SoSWeightedAvgOpponentsORebPerGame': 0.0, 
                'SoSWeightedAvgOpponentsORebAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsDRebPerGame': 0.0, 
                'SoSWeightedAvgOpponentsDRebAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsAstPerGame': 0.0, 
                'SoSWeightedAvgOpponentsAstAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsTOPerGame': 0.0, 
                'SoSWeightedAvgOpponentsTOAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsAst/TORatio': 0.0, 
                'SoSWeightedAvgOpponentsAst/TORatioAllowed': 0.0, 'SoSWeightedAvgOpponentsAst/TORatio(alt)': 0.0, 
                'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed': 0.0, 'SoSWeightedAvgOpponentsStlPerGame': 0.0, 
                'SoSWeightedAvgOpponentsStlAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsBlkPerGame': 0.0, 
                'SoSWeightedAvgOpponentsBlkAllowedPerGame': 0.0, 'SoSWeightedAvgOpponentsPFPerGame': 0.0, 
                'SoSWeightedAvgOpponentsPFAllowedPerGame': 0.0
            }

    # if this is the first game of a new day
    if rawGamesDF.iloc[index, 1] > dayNum:
        dayNum = rawGamesDF.iloc[index, 1]
        print('Season:', int(season), '   Day Number:', int(dayNum), '   Current Time:', str(datetime.now())[:-7])
        # update season totals
        startOfDaySeasonTotals = upToMinuteSeasonTotals.copy()
        # de-fragment upToMinuteSeasonTotals
        upToMinuteSeasonTotals = upToMinuteSeasonTotals.copy()

    # update winning team's up-to-the-minute season totals with results of game
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Wins'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'CloseGames'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FlipCloseWins'] += 1 - rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'WinCloseWins'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'LoseCloseWins'] += 1 - rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Points'] += rawGamesDF.loc[index, 'WScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PointsAllowed'] += rawGamesDF.loc[index, 'LScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] += rawGamesDF.loc[index, 'WFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] += rawGamesDF.loc[index, 'LFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] += rawGamesDF.loc[index, 'WFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] += rawGamesDF.loc[index, 'LFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] + rawGamesDF.loc[index, 'WFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] + rawGamesDF.loc[index, 'WFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] + rawGamesDF.loc[index, 'LFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] + rawGamesDF.loc[index, 'LFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] += rawGamesDF.loc[index, 'WFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] += rawGamesDF.loc[index, 'LFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] += rawGamesDF.loc[index, 'WFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] += rawGamesDF.loc[index, 'LFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] + rawGamesDF.loc[index, 'WFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] + rawGamesDF.loc[index, 'WFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] + rawGamesDF.loc[index, 'LFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] + rawGamesDF.loc[index, 'LFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] += rawGamesDF.loc[index, 'WFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] += rawGamesDF.loc[index, 'LFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] += rawGamesDF.loc[index, 'WFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] += rawGamesDF.loc[index, 'LFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] + rawGamesDF.loc[index, 'WFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] + rawGamesDF.loc[index, 'WFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] + rawGamesDF.loc[index, 'LFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] + rawGamesDF.loc[index, 'LFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'OReb'] += rawGamesDF.loc[index, 'WOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ORebAllowed'] += rawGamesDF.loc[index, 'LOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DReb'] += rawGamesDF.loc[index, 'WDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DRebAllowed'] += rawGamesDF.loc[index, 'LDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] += rawGamesDF.loc[index, 'WAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] += rawGamesDF.loc[index, 'LAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] += rawGamesDF.loc[index, 'WTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] += rawGamesDF.loc[index, 'LTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] + rawGamesDF.loc[index, 'WAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] + rawGamesDF.loc[index, 'WTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] + rawGamesDF.loc[index, 'LAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] + rawGamesDF.loc[index, 'LTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'WAst'] / rawGamesDF.loc[index, 'WTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'LAst'] / rawGamesDF.loc[index, 'LTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Stl'] += rawGamesDF.loc[index, 'WStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'StlAllowed'] += rawGamesDF.loc[index, 'LStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Blk'] += rawGamesDF.loc[index, 'WBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'BlkAllowed'] += rawGamesDF.loc[index, 'LBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PF'] += rawGamesDF.loc[index, 'WPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PFAllowed'] += rawGamesDF.loc[index, 'LPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ListOpponentsTeamID'].append(rawGamesDF.loc[index, 'LTeamID'])
    if startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']:
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] += GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Wins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FlipCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'WinCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'LoseCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Points'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PointsAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'OReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ORebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DRebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Stl'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'StlAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Blk'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'BlkAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PF'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PFAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed']])
    # update losing team's up-to-the-minute season totals with results of game
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] += 1
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'CloseGames'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FlipCloseWins'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'WinCloseWins'] += rawGamesDF.loc[index, 'Close']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Points'] += rawGamesDF.loc[index, 'LScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PointsAllowed'] += rawGamesDF.loc[index, 'WScore']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] += rawGamesDF.loc[index, 'LFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] += rawGamesDF.loc[index, 'WFGM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] += rawGamesDF.loc[index, 'LFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] += rawGamesDF.loc[index, 'WFGA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM'] + rawGamesDF.loc[index, 'LFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA'] + rawGamesDF.loc[index, 'LFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGMAllowed'] + rawGamesDF.loc[index, 'WFGM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGAAllowed'] + rawGamesDF.loc[index, 'WFGA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] += rawGamesDF.loc[index, 'LFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] += rawGamesDF.loc[index, 'WFGM3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] += rawGamesDF.loc[index, 'LFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] += rawGamesDF.loc[index, 'WFGA3']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3'] + rawGamesDF.loc[index, 'LFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3'] + rawGamesDF.loc[index, 'LFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGM3Allowed'] + rawGamesDF.loc[index, 'WFGM3']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FGA3Allowed'] + rawGamesDF.loc[index, 'WFGA3'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] += rawGamesDF.loc[index, 'LFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] += rawGamesDF.loc[index, 'WFTM']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] += rawGamesDF.loc[index, 'LFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] += rawGamesDF.loc[index, 'WFTA']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTM'] + rawGamesDF.loc[index, 'LFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTA'] + rawGamesDF.loc[index, 'LFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTMAllowed'] + rawGamesDF.loc[index, 'WFTM']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'FTAAllowed'] + rawGamesDF.loc[index, 'WFTA'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'OReb'] += rawGamesDF.loc[index, 'LOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ORebAllowed'] += rawGamesDF.loc[index, 'WOR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DReb'] += rawGamesDF.loc[index, 'LDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'DRebAllowed'] += rawGamesDF.loc[index, 'WDR']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] += rawGamesDF.loc[index, 'LAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] += rawGamesDF.loc[index, 'WAst']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] += rawGamesDF.loc[index, 'LTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] += rawGamesDF.loc[index, 'WTO']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Ast'] + rawGamesDF.loc[index, 'LAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TO'] + rawGamesDF.loc[index, 'LTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AstAllowed'] + rawGamesDF.loc[index, 'WAst']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'TOAllowed'] + rawGamesDF.loc[index, 'WTO'])
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'LAst'] / rawGamesDF.loc[index, 'LTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'AvgAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + rawGamesDF.loc[index, 'WAst'] / rawGamesDF.loc[index, 'WTO']) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] + 1)
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Stl'] += rawGamesDF.loc[index, 'LStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'StlAllowed'] += rawGamesDF.loc[index, 'WStl']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'Blk'] += rawGamesDF.loc[index, 'LBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'BlkAllowed'] += rawGamesDF.loc[index, 'WBlk']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PF'] += rawGamesDF.loc[index, 'LPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'PFAllowed'] += rawGamesDF.loc[index, 'WPF']
    upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'ListOpponentsTeamID'].append(rawGamesDF.loc[index, 'WTeamID'])
    if startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']:
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] += GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Wins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFlipCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FlipCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsWinCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'WinCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsLoseCloseWinPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'LoseCloseWins'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Points'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPointsAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PointsAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGM3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3PerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFGA3AllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3Pct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFG3PctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGM3Allowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FGA3Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTMAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTAAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPct'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPct'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTM'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTA'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsFTPctAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTMAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'FTAAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'OReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsORebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'ORebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DReb'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsDRebAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'DRebAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAstAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsTOAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Ast'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TO'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatioAllowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AstAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'TOAllowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'AvgAst/TORatio(alt)Allowed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Stl'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsStlAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'StlAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'Blk'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsBlkAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'BlkAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PF'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])
        upToMinuteSeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] = (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSWeightedAvgOpponentsPFAllowedPerGame'] * startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'PFAllowed'] / startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] * GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']]) / (startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'SoSSumOppGamesPlayedWeightsDenominator'] + GamesPlayedWeights[startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed']])

    # if both teams have played at least 5 games prior to this one during the current season, and both teams have at least 1 season of historical data
    if startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'WTeamID']), 'GamesPlayed'] >= 5 and startOfDaySeasonTotals.loc[int(rawGamesDF.loc[index, 'LTeamID']), 'GamesPlayed'] >= 5 \
            and seasonTotalsDict[season - 1].loc[rawGamesDF.loc[index, 'WTeamID'], 'GamesPlayed'] and seasonTotalsDict[season - 1].loc[rawGamesDF.loc[index, 'LTeamID'], 'GamesPlayed']:

        # create input instance for game

        # record Season and DayNum
        inputData.loc[index, 'Season'] = season
        inputData.loc[index, 'DayNum'] = dayNum
        
        # flip flag to alternate winning team between Team A and Team B each game
        TeamAWins = not TeamAWins
        if TeamAWins:
            teamA = rawGamesDF.loc[index, 'WTeamID']
            teamB = rawGamesDF.loc[index, 'LTeamID']
        else:
            teamA = rawGamesDF.loc[index, 'LTeamID']
            teamB = rawGamesDF.loc[index, 'WTeamID']
            
        # assign TeamAWins (True if Team A wins, False if Team B wins)
        inputData.loc[index, 'TeamAWins'] = TeamAWins

        # use teams' four-digit identification numbers as pseudo-indexes
        inputData.loc[index, 'TeamID A'] = teamA
        inputData.loc[index, 'TeamID B'] = teamB

        # Team A's features
        inputData.loc[index, 'HomeAdv A'] = (2 * TeamAWins - 1) * HomeAdvDict[rawGamesDF.loc[index, 'WLoc']]
        inputData.loc[index, 'NumGamesPlayedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']

        inputData.loc[index, 'WinPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'Wins'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'WinPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'WinPct']

        inputData.loc[index, 'FlipCloseWinPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FlipCloseWins'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FlipCloseWinPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FlipCloseWinPct']

        inputData.loc[index, 'WinCloseWinPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'WinCloseWins'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'WinCloseWinPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'WinCloseWinPct']

        inputData.loc[index, 'LoseCloseWinPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'LoseCloseWins'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'LoseCloseWinPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'LoseCloseWinPct']

        inputData.loc[index, 'PointsPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'Points'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PointsPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PointsPerGame']

        inputData.loc[index, 'PointsAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'PointsAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PointsAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PointsAllowedPerGame']

        inputData.loc[index, 'PointsMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'Points'] - startOfDaySeasonTotals.loc[teamA, 'PointsAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PointsMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PointsMarginPerGame']

        inputData.loc[index, 'FGMPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGMPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGMPerGame']

        inputData.loc[index, 'FGMAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGMAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGMAllowedPerGame']

        inputData.loc[index, 'FGMMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FGM'] - startOfDaySeasonTotals.loc[teamA, 'FGMAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGMMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGMMarginPerGame']

        inputData.loc[index, 'FGAPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGA'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGAPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGAPerGame']

        inputData.loc[index, 'FGAAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGAAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGAAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGAAllowedPerGame']

        inputData.loc[index, 'FGAMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FGA'] - startOfDaySeasonTotals.loc[teamA, 'FGAAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGAMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGAMarginPerGame']

        inputData.loc[index, 'FGPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM'] / startOfDaySeasonTotals.loc[teamA, 'FGA']
        inputData.loc[index, 'FGPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGPct']

        inputData.loc[index, 'FGPctAllowedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'FGAAllowed']
        inputData.loc[index, 'FGPctAllowedLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGPctAllowed']

        inputData.loc[index, 'FGPctMarginThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM'] / startOfDaySeasonTotals.loc[teamA, 'FGA'] - startOfDaySeasonTotals.loc[teamA, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'FGAAllowed']
        inputData.loc[index, 'FGPctMarginLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGPctMargin']

        inputData.loc[index, 'FGM3PerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM3'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGM3PerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGM3PerGame']

        inputData.loc[index, 'FGM3AllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGM3AllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGM3AllowedPerGame']

        inputData.loc[index, 'FGM3MarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FGM3'] - startOfDaySeasonTotals.loc[teamA, 'FGM3Allowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGM3MarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGM3MarginPerGame']

        inputData.loc[index, 'FGA3PerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGA3'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGA3PerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGA3PerGame']

        inputData.loc[index, 'FGA3AllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGA3Allowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGA3AllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGA3AllowedPerGame']

        inputData.loc[index, 'FGA3MarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FGA3'] - startOfDaySeasonTotals.loc[teamA, 'FGA3Allowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FGA3MarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FGA3MarginPerGame']

        inputData.loc[index, 'FG3PctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM3'] / startOfDaySeasonTotals.loc[teamA, 'FGA3']
        inputData.loc[index, 'FG3PctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FG3Pct']

        inputData.loc[index, 'FG3PctAllowedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamA, 'FGA3Allowed']
        inputData.loc[index, 'FG3PctAllowedLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FG3PctAllowed']

        inputData.loc[index, 'FG3PctMarginThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FGM3'] / startOfDaySeasonTotals.loc[teamA, 'FGA3'] - startOfDaySeasonTotals.loc[teamA, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamA, 'FGA3Allowed']
        inputData.loc[index, 'FG3PctMarginLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FG3PctMargin']

        inputData.loc[index, 'FTMPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTM'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTMPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTMPerGame']

        inputData.loc[index, 'FTMAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTMAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTMAllowedPerGame']

        inputData.loc[index, 'FTMMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FTM'] - startOfDaySeasonTotals.loc[teamA, 'FTMAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTMMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTMMarginPerGame']

        inputData.loc[index, 'FTAPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTA'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTAPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTAPerGame']

        inputData.loc[index, 'FTAAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTAAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTAAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTAAllowedPerGame']

        inputData.loc[index, 'FTAMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'FTA'] - startOfDaySeasonTotals.loc[teamA, 'FTAAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'FTAMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTAMarginPerGame']

        inputData.loc[index, 'FTPctThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTM'] / startOfDaySeasonTotals.loc[teamA, 'FTA']
        inputData.loc[index, 'FTPctLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTPct']

        inputData.loc[index, 'FTPctAllowedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'FTAAllowed']
        inputData.loc[index, 'FTPctAllowedLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTPctAllowed']

        inputData.loc[index, 'FTPctMarginThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'FTM'] / startOfDaySeasonTotals.loc[teamA, 'FTA'] - startOfDaySeasonTotals.loc[teamA, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamA, 'FTAAllowed']
        inputData.loc[index, 'FTPctMarginLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'FTPctMargin']

        inputData.loc[index, 'RebPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'OReb'] + startOfDaySeasonTotals.loc[teamA, 'DReb']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'RebPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'RebPerGame']

        inputData.loc[index, 'RebAllowedPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'ORebAllowed'] + startOfDaySeasonTotals.loc[teamA, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'RebAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'RebAllowedPerGame']

        inputData.loc[index, 'RebMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'OReb'] + startOfDaySeasonTotals.loc[teamA, 'DReb'] - startOfDaySeasonTotals.loc[teamA, 'ORebAllowed'] - startOfDaySeasonTotals.loc[teamA, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'RebMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'RebMarginPerGame']

        inputData.loc[index, 'ORebPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'OReb'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'ORebPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'ORebPerGame']

        inputData.loc[index, 'ORebAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'ORebAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'ORebAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'ORebAllowedPerGame']

        inputData.loc[index, 'ORebMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'OReb'] - startOfDaySeasonTotals.loc[teamA, 'ORebAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'ORebMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'ORebMarginPerGame']

        inputData.loc[index, 'DRebPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'DReb'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'DRebPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'DRebPerGame']

        inputData.loc[index, 'DRebAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'DRebAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'DRebAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'DRebAllowedPerGame']

        inputData.loc[index, 'DRebMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'DReb'] - startOfDaySeasonTotals.loc[teamA, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'DRebMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'DRebMarginPerGame']

        inputData.loc[index, 'AstPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'Ast'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'AstPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AstPerGame']

        inputData.loc[index, 'AstAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AstAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'AstAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AstAllowedPerGame']

        inputData.loc[index, 'AstMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'Ast'] - startOfDaySeasonTotals.loc[teamA, 'AstAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'AstMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AstMarginPerGame']

        inputData.loc[index, 'TOPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'TO'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'TOPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'TOPerGame']

        inputData.loc[index, 'TOAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'TOAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'TOAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'TOAllowedPerGame']

        inputData.loc[index, 'TOMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'TO'] - startOfDaySeasonTotals.loc[teamA, 'TOAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'TOMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'TOMarginPerGame']

        inputData.loc[index, 'AvgAst/TORatioThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio']
        inputData.loc[index, 'AvgAst/TORatioLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatio']

        inputData.loc[index, 'AvgAst/TORatioAllowedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatioAllowed']
        inputData.loc[index, 'AvgAst/TORatioAllowedLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatioAllowed']

        inputData.loc[index, 'AvgAst/TORatioMarginThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio'] - startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatioAllowed']
        inputData.loc[index, 'AvgAst/TORatioMarginLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatioMargin']

        inputData.loc[index, 'AvgAst/TORatio(alt)ThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio(alt)']
        inputData.loc[index, 'AvgAst/TORatio(alt)LastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatio(alt)']

        inputData.loc[index, 'AvgAst/TORatio(alt)AllowedThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio(alt)Allowed']
        inputData.loc[index, 'AvgAst/TORatio(alt)AllowedLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatio(alt)Allowed']

        inputData.loc[index, 'AvgAst/TORatio(alt)MarginThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[teamA, 'AvgAst/TORatio(alt)Allowed']
        inputData.loc[index, 'AvgAst/TORatio(alt)MarginLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'AvgAst/TORatio(alt)Margin']

        inputData.loc[index, 'StlPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'Stl'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'StlPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'StlPerGame']

        inputData.loc[index, 'StlAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'StlAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'StlAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'StlAllowedPerGame']

        inputData.loc[index, 'StlMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'Stl'] - startOfDaySeasonTotals.loc[teamA, 'StlAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'StlMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'StlMarginPerGame']

        inputData.loc[index, 'BlkPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'Blk'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'BlkPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'BlkPerGame']

        inputData.loc[index, 'BlkAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'BlkAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'BlkAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'BlkAllowedPerGame']

        inputData.loc[index, 'BlkMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'Blk'] - startOfDaySeasonTotals.loc[teamA, 'BlkAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'BlkMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'BlkMarginPerGame']

        inputData.loc[index, 'PFPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'PF'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PFPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PFPerGame']

        inputData.loc[index, 'PFAllowedPerGameThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'PFAllowed'] / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PFAllowedPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PFAllowedPerGame']

        inputData.loc[index, 'PFMarginPerGameThisSeason A'] = (startOfDaySeasonTotals.loc[teamA, 'PF'] - startOfDaySeasonTotals.loc[teamA, 'PFAllowed']) / startOfDaySeasonTotals.loc[teamA, 'GamesPlayed']
        inputData.loc[index, 'PFMarginPerGameLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'PFMarginPerGame']

        # strength of schedule

        inputData.loc[index, 'CurrentSoSWeightedAvgOppWinPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppWinPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppWinPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Wins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppWinPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Wins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppWinPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFlipCloseWinPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFlipCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFlipCloseWinPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFlipCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppFlipCloseWinPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FlipCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFlipCloseWinPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FlipCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFlipCloseWinPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFlipCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppWinCloseWinPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsWinCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppWinCloseWinPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsWinCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppWinCloseWinPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'WinCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppWinCloseWinPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'WinCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppWinCloseWinPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppWinCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppLoseCloseWinPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsLoseCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppLoseCloseWinPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsLoseCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppLoseCloseWinPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'LoseCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppLoseCloseWinPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'LoseCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppLoseCloseWinPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppLoseCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPointsPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPointsPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Points'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Points'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPointsPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PointsAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PointsAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPointsAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPointsPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPointsPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Points'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PointsAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Points'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PointsAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPointsMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGMPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGMPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGMPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGMAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGMPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGMPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGMMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGAPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGAPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGAPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGAAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGAPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGAPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGAMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGPct']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctAllowedThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctAllowedThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctAllowedNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctAllowedNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGAAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctAllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGPctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctMarginThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGPct'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctMarginThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGPct'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctMarginNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGPct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGPctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctMarginNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGPct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGPctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctMarginNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGPctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3PerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGM3PerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3PerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGM3PerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3PerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3PerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3PerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGM3PerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3AllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3AllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3AllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3AllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3AllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGM3AllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3MarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGM3PerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3MarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGM3PerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3MarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3MarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3MarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGM3MarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3PerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGA3PerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3PerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGA3PerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3PerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3PerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3PerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGA3PerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3AllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3AllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3AllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3AllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3AllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGA3AllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3MarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGA3PerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3MarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGA3PerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3MarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3MarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3MarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFGA3MarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFG3Pct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFG3Pct']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFG3PctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctAllowedThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctAllowedThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctAllowedNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctAllowedNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGM3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FGA3Allowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctAllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFG3PctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctMarginThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFG3Pct'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctMarginThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFG3Pct'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctMarginNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FG3Pct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FG3PctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctMarginNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FG3Pct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FG3PctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctMarginNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFG3PctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTMPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTMPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTMPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTMAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTMPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTMPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTMMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTAPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTAPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTAPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTAAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTAPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTAPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTAMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTPct']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTA']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctAllowedThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctAllowedThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctAllowedNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctAllowedNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTAAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctAllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTPctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctMarginThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTPct'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctMarginThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTPct'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctMarginNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTPct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTPctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctMarginNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTPct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'FTPctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctMarginNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppFTPctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] + startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] + seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppRebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] + startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] + seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebAllowedPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebAllowedPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppRebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] + startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] + seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppRebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppORebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppORebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'OReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'ORebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppORebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppDRebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppDRebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppDRebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAstPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAstPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Ast'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Ast'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAstPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AstAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AstAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAstAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAstPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAstPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Ast'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AstAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Ast'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AstAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAstMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsTOPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsTOPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TO'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TO'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppTOPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TOAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TOAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppTOAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsTOPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsTOPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TO'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TOAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TO'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'TOAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppTOMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioNowThisSeason A'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioNowLastSeason A'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatioNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioAllowedThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioAllowedThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioAllowedNowThisSeason A'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioAllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioAllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatioAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioMarginThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioMarginThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioMarginNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioMarginNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioMarginNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatioMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)ThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)ThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)NowThisSeason A'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)NowLastSeason A'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)NowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatio(alt)Now']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowThisSeason A'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatio(alt)AllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppAvgAst/TORatio(alt)MarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsStlPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsStlPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Stl'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Stl'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppStlPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'StlAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'StlAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppStlAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsStlPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsStlPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Stl'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'StlAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Stl'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'StlAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppStlMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsBlkPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsBlkPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Blk'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Blk'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppBlkPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'BlkAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'BlkAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppBlkAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsBlkPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsBlkPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Blk'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'BlkAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'Blk'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'BlkAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppBlkMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPFPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPFPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PF'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PF'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPFPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFAllowedPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFAllowedPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFAllowedPerGameNowThisSeason A'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PFAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFAllowedPerGameNowLastSeason A'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PFAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFAllowedPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPFAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFMarginPerGameThenThisSeason A'] = startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPFPerGame'] - startOfDaySeasonTotals.loc[teamA, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFMarginPerGameThenLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPFPerGame'] - seasonTotalsDict[season - 1].loc[teamA, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFMarginPerGameNowThisSeason A'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PF'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PFAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFMarginPerGameNowLastSeason A'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PF'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'PFAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamA, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFMarginPerGameNowLastSeason A'] = seasonTotalsDict[season - 1].loc[teamA, 'SoSAvgOppPFMarginPerGameNow']

        # Team B's features

        inputData.loc[index, 'NumGamesPlayedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']

        inputData.loc[index, 'WinPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'Wins'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'WinPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'WinPct']

        inputData.loc[index, 'FlipCloseWinPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FlipCloseWins'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FlipCloseWinPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FlipCloseWinPct']

        inputData.loc[index, 'WinCloseWinPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'WinCloseWins'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'WinCloseWinPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'WinCloseWinPct']

        inputData.loc[index, 'LoseCloseWinPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'LoseCloseWins'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'LoseCloseWinPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'LoseCloseWinPct']

        inputData.loc[index, 'PointsPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'Points'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PointsPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PointsPerGame']

        inputData.loc[index, 'PointsAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'PointsAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PointsAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PointsAllowedPerGame']

        inputData.loc[index, 'PointsMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'Points'] - startOfDaySeasonTotals.loc[teamB, 'PointsAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PointsMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PointsMarginPerGame']

        inputData.loc[index, 'FGMPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGMPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGMPerGame']

        inputData.loc[index, 'FGMAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGMAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGMAllowedPerGame']

        inputData.loc[index, 'FGMMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FGM'] - startOfDaySeasonTotals.loc[teamB, 'FGMAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGMMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGMMarginPerGame']

        inputData.loc[index, 'FGAPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGA'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGAPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGAPerGame']

        inputData.loc[index, 'FGAAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGAAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGAAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGAAllowedPerGame']

        inputData.loc[index, 'FGAMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FGA'] - startOfDaySeasonTotals.loc[teamB, 'FGAAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGAMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGAMarginPerGame']

        inputData.loc[index, 'FGPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM'] / startOfDaySeasonTotals.loc[teamB, 'FGA']
        inputData.loc[index, 'FGPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGPct']

        inputData.loc[index, 'FGPctAllowedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'FGAAllowed']
        inputData.loc[index, 'FGPctAllowedLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGPctAllowed']

        inputData.loc[index, 'FGPctMarginThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM'] / startOfDaySeasonTotals.loc[teamB, 'FGA'] - startOfDaySeasonTotals.loc[teamB, 'FGMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'FGAAllowed']
        inputData.loc[index, 'FGPctMarginLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGPctMargin']

        inputData.loc[index, 'FGM3PerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM3'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGM3PerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGM3PerGame']

        inputData.loc[index, 'FGM3AllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGM3AllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGM3AllowedPerGame']

        inputData.loc[index, 'FGM3MarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FGM3'] - startOfDaySeasonTotals.loc[teamB, 'FGM3Allowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGM3MarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGM3MarginPerGame']

        inputData.loc[index, 'FGA3PerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGA3'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGA3PerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGA3PerGame']

        inputData.loc[index, 'FGA3AllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGA3Allowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGA3AllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGA3AllowedPerGame']

        inputData.loc[index, 'FGA3MarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FGA3'] - startOfDaySeasonTotals.loc[teamB, 'FGA3Allowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FGA3MarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FGA3MarginPerGame']

        inputData.loc[index, 'FG3PctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM3'] / startOfDaySeasonTotals.loc[teamB, 'FGA3']
        inputData.loc[index, 'FG3PctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FG3Pct']

        inputData.loc[index, 'FG3PctAllowedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamB, 'FGA3Allowed']
        inputData.loc[index, 'FG3PctAllowedLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FG3PctAllowed']

        inputData.loc[index, 'FG3PctMarginThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FGM3'] / startOfDaySeasonTotals.loc[teamB, 'FGA3'] - startOfDaySeasonTotals.loc[teamB, 'FGM3Allowed'] / startOfDaySeasonTotals.loc[teamB, 'FGA3Allowed']
        inputData.loc[index, 'FG3PctMarginLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FG3PctMargin']

        inputData.loc[index, 'FTMPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTM'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTMPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTMPerGame']

        inputData.loc[index, 'FTMAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTMAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTMAllowedPerGame']

        inputData.loc[index, 'FTMMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FTM'] - startOfDaySeasonTotals.loc[teamB, 'FTMAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTMMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTMMarginPerGame']

        inputData.loc[index, 'FTAPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTA'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTAPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTAPerGame']

        inputData.loc[index, 'FTAAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTAAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTAAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTAAllowedPerGame']

        inputData.loc[index, 'FTAMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'FTA'] - startOfDaySeasonTotals.loc[teamB, 'FTAAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'FTAMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTAMarginPerGame']

        inputData.loc[index, 'FTPctThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTM'] / startOfDaySeasonTotals.loc[teamB, 'FTA']
        inputData.loc[index, 'FTPctLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTPct']

        inputData.loc[index, 'FTPctAllowedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'FTAAllowed']
        inputData.loc[index, 'FTPctAllowedLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTPctAllowed']

        inputData.loc[index, 'FTPctMarginThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'FTM'] / startOfDaySeasonTotals.loc[teamB, 'FTA'] - startOfDaySeasonTotals.loc[teamB, 'FTMAllowed'] / startOfDaySeasonTotals.loc[teamB, 'FTAAllowed']
        inputData.loc[index, 'FTPctMarginLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'FTPctMargin']

        inputData.loc[index, 'RebPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'OReb'] + startOfDaySeasonTotals.loc[teamB, 'DReb']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'RebPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'RebPerGame']

        inputData.loc[index, 'RebAllowedPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'ORebAllowed'] + startOfDaySeasonTotals.loc[teamB, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'RebAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'RebAllowedPerGame']

        inputData.loc[index, 'RebMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'OReb'] + startOfDaySeasonTotals.loc[teamB, 'DReb'] - startOfDaySeasonTotals.loc[teamB, 'ORebAllowed'] - startOfDaySeasonTotals.loc[teamB, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'RebMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'RebMarginPerGame']

        inputData.loc[index, 'ORebPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'OReb'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'ORebPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'ORebPerGame']

        inputData.loc[index, 'ORebAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'ORebAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'ORebAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'ORebAllowedPerGame']

        inputData.loc[index, 'ORebMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'OReb'] - startOfDaySeasonTotals.loc[teamB, 'ORebAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'ORebMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'ORebMarginPerGame']

        inputData.loc[index, 'DRebPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'DReb'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'DRebPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'DRebPerGame']

        inputData.loc[index, 'DRebAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'DRebAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'DRebAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'DRebAllowedPerGame']

        inputData.loc[index, 'DRebMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'DReb'] - startOfDaySeasonTotals.loc[teamB, 'DRebAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'DRebMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'DRebMarginPerGame']

        inputData.loc[index, 'AstPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'Ast'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'AstPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AstPerGame']

        inputData.loc[index, 'AstAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AstAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'AstAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AstAllowedPerGame']

        inputData.loc[index, 'AstMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'Ast'] - startOfDaySeasonTotals.loc[teamB, 'AstAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'AstMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AstMarginPerGame']

        inputData.loc[index, 'TOPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'TO'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'TOPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'TOPerGame']

        inputData.loc[index, 'TOAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'TOAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'TOAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'TOAllowedPerGame']

        inputData.loc[index, 'TOMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'TO'] - startOfDaySeasonTotals.loc[teamB, 'TOAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'TOMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'TOMarginPerGame']

        inputData.loc[index, 'AvgAst/TORatioThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio']
        inputData.loc[index, 'AvgAst/TORatioLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatio']

        inputData.loc[index, 'AvgAst/TORatioAllowedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatioAllowed']
        inputData.loc[index, 'AvgAst/TORatioAllowedLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatioAllowed']

        inputData.loc[index, 'AvgAst/TORatioMarginThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio'] - startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatioAllowed']
        inputData.loc[index, 'AvgAst/TORatioMarginLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatioMargin']

        inputData.loc[index, 'AvgAst/TORatio(alt)ThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio(alt)']
        inputData.loc[index, 'AvgAst/TORatio(alt)LastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatio(alt)']

        inputData.loc[index, 'AvgAst/TORatio(alt)AllowedThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio(alt)Allowed']
        inputData.loc[index, 'AvgAst/TORatio(alt)AllowedLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatio(alt)Allowed']

        inputData.loc[index, 'AvgAst/TORatio(alt)MarginThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[teamB, 'AvgAst/TORatio(alt)Allowed']
        inputData.loc[index, 'AvgAst/TORatio(alt)MarginLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'AvgAst/TORatio(alt)Margin']

        inputData.loc[index, 'StlPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'Stl'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'StlPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'StlPerGame']

        inputData.loc[index, 'StlAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'StlAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'StlAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'StlAllowedPerGame']

        inputData.loc[index, 'StlMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'Stl'] - startOfDaySeasonTotals.loc[teamB, 'StlAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'StlMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'StlMarginPerGame']

        inputData.loc[index, 'BlkPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'Blk'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'BlkPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'BlkPerGame']

        inputData.loc[index, 'BlkAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'BlkAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'BlkAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'BlkAllowedPerGame']

        inputData.loc[index, 'BlkMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'Blk'] - startOfDaySeasonTotals.loc[teamB, 'BlkAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'BlkMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'BlkMarginPerGame']

        inputData.loc[index, 'PFPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'PF'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PFPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PFPerGame']

        inputData.loc[index, 'PFAllowedPerGameThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'PFAllowed'] / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PFAllowedPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PFAllowedPerGame']

        inputData.loc[index, 'PFMarginPerGameThisSeason B'] = (startOfDaySeasonTotals.loc[teamB, 'PF'] - startOfDaySeasonTotals.loc[teamB, 'PFAllowed']) / startOfDaySeasonTotals.loc[teamB, 'GamesPlayed']
        inputData.loc[index, 'PFMarginPerGameLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'PFMarginPerGame']

        # strength of schedule

        inputData.loc[index, 'CurrentSoSWeightedAvgOppWinPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppWinPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppWinPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Wins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppWinPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Wins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppWinPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFlipCloseWinPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFlipCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFlipCloseWinPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFlipCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppFlipCloseWinPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FlipCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFlipCloseWinPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FlipCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFlipCloseWinPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFlipCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppWinCloseWinPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsWinCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppWinCloseWinPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsWinCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppWinCloseWinPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'WinCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppWinCloseWinPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'WinCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppWinCloseWinPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppWinCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppLoseCloseWinPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsLoseCloseWinPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppLoseCloseWinPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsLoseCloseWinPct']
        inputData.loc[index, 'CurrentSoSAvgOppLoseCloseWinPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'LoseCloseWins'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppLoseCloseWinPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'LoseCloseWins'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppLoseCloseWinPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppLoseCloseWinPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPointsPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPointsPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Points'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Points'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPointsPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PointsAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PointsAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPointsAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPointsMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPointsPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPointsMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPointsPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPointsAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPointsMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Points'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PointsAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPointsMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Points'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PointsAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPointsMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPointsMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGMPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGMPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGMPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGMAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGMMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGMPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGMMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGMPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGMMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGMMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGMMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGMMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGAPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGAPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGAPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGAAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGAMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGAPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGAMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGAPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGAMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGAMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGAMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGAMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGPct']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctAllowedThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctAllowedThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctAllowedNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctAllowedNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGAAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctAllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGPctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGPctMarginThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGPct'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGPctMarginThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGPct'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFGPctMarginNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGPct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGPctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGPctMarginNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGPct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGPctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGPctMarginNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGPctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3PerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGM3PerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3PerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGM3PerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3PerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3PerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3PerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGM3PerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3AllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3AllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3AllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3AllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3AllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGM3AllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGM3MarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGM3PerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGM3MarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGM3PerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGM3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGM3MarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGM3MarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGM3MarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGM3MarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3PerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGA3PerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3PerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGA3PerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3PerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3PerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3PerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGA3PerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3AllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3AllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3AllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3AllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3AllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGA3AllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFGA3MarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGA3PerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFGA3MarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGA3PerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFGA3AllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFGA3MarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFGA3MarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFGA3MarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFGA3MarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFG3Pct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFG3Pct']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFG3PctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctAllowedThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctAllowedThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctAllowedNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctAllowedNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGM3Allowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FGA3Allowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctAllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFG3PctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFG3PctMarginThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFG3Pct'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFG3PctMarginThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFG3Pct'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFG3PctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctMarginNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FG3Pct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FG3PctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFG3PctMarginNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FG3Pct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FG3PctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFG3PctMarginNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFG3PctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTMPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTMPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTMPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTMAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTMMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTMPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTMMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTMPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTMAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTMMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTMMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTMMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTMMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTAPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTAPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTAPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTAAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTAMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTAPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTAMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTAPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTAAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppFTAMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTAMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTAMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTAMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTPct']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTPct']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTM'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTA']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTPctNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctAllowedThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctAllowedThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctAllowedNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctAllowedNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTMAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTAAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctAllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTPctAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppFTPctMarginThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTPct'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppFTPctMarginThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTPct'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsFTPctAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppFTPctMarginNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTPct'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTPctAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppFTPctMarginNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTPct'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'FTPctAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppFTPctMarginNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppFTPctMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] + startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] + seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppRebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] + startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] + seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebAllowedPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebAllowedPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppRebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppRebMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] + startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppRebMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] + seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppRebMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] + startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppRebMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] + seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppRebMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppRebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppORebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppORebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppORebMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppORebMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsORebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppORebMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppORebMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'OReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'ORebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppORebMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppORebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppDRebPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppDRebAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppDRebMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppDRebMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsDRebAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppDRebMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppDRebMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DReb'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'DRebAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppDRebMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppDRebMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAstPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAstPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Ast'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Ast'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAstPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AstAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AstAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAstAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAstMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAstPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppAstMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAstPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAstAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppAstMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Ast'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AstAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAstMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Ast'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AstAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAstMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAstMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsTOPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsTOPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TO'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TO'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppTOPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TOAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TOAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppTOAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppTOMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsTOPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppTOMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsTOPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsTOAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppTOMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TO'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TOAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppTOMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TO'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'TOAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppTOMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppTOMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioNowThisSeason B'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioNowLastSeason B'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatioNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioAllowedThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioAllowedThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioAllowedNowThisSeason B'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioAllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioAllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatioAllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatioMarginThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatioMarginThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatioAllowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioMarginNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatioMarginNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatioAllowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatioMarginNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatioMarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)ThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)ThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)NowThisSeason B'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)NowLastSeason B'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)NowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatio(alt)Now']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)AllowedThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowThisSeason B'] = startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed'].mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)AllowedNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatio(alt)AllowedNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'PastSoSWeightedAvgOppAvgAst/TORatio(alt)MarginThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsAst/TORatio(alt)Allowed']
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'AvgAst/TORatio(alt)Allowed']).mean()
        inputData.loc[index, 'PastSoSAvgOppAvgAst/TORatio(alt)MarginNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppAvgAst/TORatio(alt)MarginNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsStlPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsStlPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Stl'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Stl'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppStlPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'StlAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'StlAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppStlAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppStlMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsStlPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppStlMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsStlPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsStlAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppStlMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Stl'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'StlAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppStlMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Stl'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'StlAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppStlMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppStlMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsBlkPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsBlkPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Blk'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Blk'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppBlkPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'BlkAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'BlkAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppBlkAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppBlkMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsBlkPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppBlkMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsBlkPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsBlkAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppBlkMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Blk'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'BlkAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppBlkMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'Blk'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'BlkAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppBlkMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppBlkMarginPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPFPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPFPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PF'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PF'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPFPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFAllowedPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFAllowedPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFAllowedPerGameNowThisSeason B'] = (startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PFAllowed'] / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFAllowedPerGameNowLastSeason B'] = (seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PFAllowed'] / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFAllowedPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPFAllowedPerGameNow']

        inputData.loc[index, 'CurrentSoSWeightedAvgOppPFMarginPerGameThenThisSeason B'] = startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPFPerGame'] - startOfDaySeasonTotals.loc[teamB, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'PastSoSWeightedAvgOppPFMarginPerGameThenLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPFPerGame'] - seasonTotalsDict[season - 1].loc[teamB, 'SoSWeightedAvgOpponentsPFAllowedPerGame']
        inputData.loc[index, 'CurrentSoSAvgOppPFMarginPerGameNowThisSeason B'] = ((startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PF'] - startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PFAllowed']) / startOfDaySeasonTotals.loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'CurrentSoSAvgOppPFMarginPerGameNowLastSeason B'] = ((seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PF'] - seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'PFAllowed']) / seasonTotalsDict[season - 1].loc[startOfDaySeasonTotals.loc[teamB, 'ListOpponentsTeamID'], 'GamesPlayed']).mean()
        inputData.loc[index, 'PastSoSAvgOppPFMarginPerGameNowLastSeason B'] = seasonTotalsDict[season - 1].loc[teamB, 'SoSAvgOppPFMarginPerGameNow']


# calculate total runtime
delta = datetime.now() - startTime
print('\nTotal runtime: ' + str(delta.seconds // 3600) + ' hours, ' + str(delta.seconds // 60) + ' minutes, ' + str(delta.seconds % 60) + ' seconds')


# concatenate data from each year 
data = pd.read_csv(rootDirectory + '/Data/Training_Data/Training_Data_' + str(startingSeason + 1) + '.csv')
print(str(int(startingSeason + 1)), data.shape)
for year in range(startingSeason + 2, currentSeason):
    data = pd.concat([data, pd.read_csv(rootDirectory + '/Data/Training_Data/Training_Data_' + str(year) + '.csv')])
    print(str(int(year)), data.shape)
# drop unnecessary columns
data = data.drop(columns=['TeamID A', 'TeamID B'])
# save concatenated data
data.to_csv(rootDirectory + '/Data/Training_Data/All_Training_Data.csv', index=False)
print('All_Training_Data.csv written.')

print('Generate training data: completed.')


Season: 2004    Current time: 2024-08-23 11:50:29
Season: 2004    Day Number: 10    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 11    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 12    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 13    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 14    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 15    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 16    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 17    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 18    Current Time: 2024-08-23 11:50:31
Season: 2004    Day Number: 19    Current Time: 2024-08-23 11:50:32
Season: 2004    Day Number: 20    Current Time: 2024-08-23 11:50:32
Season: 2004    Day Number: 21    Current Time: 2024-08-23 11:50:32
Season: 2004    Day Number: 22    Current Time: 2024-08-23 11:50:33
Season: 2004    Day Number: 23    Current Time: 2024-08-23 11:50:3

SeasonTotals_2004.csv written
Training_Data_2004.csv written
Season: 2005    Current time: 2024-08-23 11:59:31
Season: 2005    Day Number: 10    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 11    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 12    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 13    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 14    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 15    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 16    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 17    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 18    Current Time: 2024-08-23 11:59:33
Season: 2005    Day Number: 19    Current Time: 2024-08-23 11:59:34
Season: 2005    Day Number: 20    Current Time: 2024-08-23 11:59:34
Season: 2005    Day Number: 21    Current Time: 2024-08-23 11:59:34
Season: 2005    Day Number: 22    Current Time: 2024-08-23 11:59:35
Seaso

Season: 2005    Day Number: 131    Current Time: 2024-08-23 12:08:23
Season: 2005    Day Number: 132    Current Time: 2024-08-23 12:08:26
SeasonTotals_2005.csv written
Training_Data_2005.csv written
Season: 2006    Current time: 2024-08-23 12:08:38
Season: 2006    Day Number: 8    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 9    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 10    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 11    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 12    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 13    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 14    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 15    Current Time: 2024-08-23 12:08:40
Season: 2006    Day Number: 16    Current Time: 2024-08-23 12:08:41
Season: 2006    Day Number: 17    Current Time: 2024-08-23 12:08:41
Season: 2006    Day Number: 18    Current Time: 2024-08-23 12:08:41
Seaso

Season: 2006    Day Number: 127    Current Time: 2024-08-23 12:17:09
Season: 2006    Day Number: 128    Current Time: 2024-08-23 12:17:11
Season: 2006    Day Number: 129    Current Time: 2024-08-23 12:17:14
Season: 2006    Day Number: 130    Current Time: 2024-08-23 12:17:22
Season: 2006    Day Number: 131    Current Time: 2024-08-23 12:17:28
Season: 2006    Day Number: 132    Current Time: 2024-08-23 12:17:31
SeasonTotals_2006.csv written
Training_Data_2006.csv written
Season: 2007    Current time: 2024-08-23 12:17:43
Season: 2007    Day Number: 8    Current Time: 2024-08-23 12:17:45
Season: 2007    Day Number: 9    Current Time: 2024-08-23 12:17:45
Season: 2007    Day Number: 10    Current Time: 2024-08-23 12:17:45
Season: 2007    Day Number: 11    Current Time: 2024-08-23 12:17:45
Season: 2007    Day Number: 12    Current Time: 2024-08-23 12:17:45
Season: 2007    Day Number: 13    Current Time: 2024-08-23 12:17:46
Season: 2007    Day Number: 14    Current Time: 2024-08-23 12:17:46
S

Season: 2007    Day Number: 124    Current Time: 2024-08-23 12:36:44
Season: 2007    Day Number: 125    Current Time: 2024-08-23 12:36:59
Season: 2007    Day Number: 126    Current Time: 2024-08-23 12:37:04
Season: 2007    Day Number: 127    Current Time: 2024-08-23 12:37:05
Season: 2007    Day Number: 128    Current Time: 2024-08-23 12:37:06
Season: 2007    Day Number: 129    Current Time: 2024-08-23 12:37:11
Season: 2007    Day Number: 130    Current Time: 2024-08-23 12:37:19
Season: 2007    Day Number: 131    Current Time: 2024-08-23 12:37:25
Season: 2007    Day Number: 132    Current Time: 2024-08-23 12:37:28
SeasonTotals_2007.csv written
Training_Data_2007.csv written
Season: 2008    Current time: 2024-08-23 12:37:40
Season: 2008    Day Number: 0    Current Time: 2024-08-23 12:37:42
Season: 2008    Day Number: 1    Current Time: 2024-08-23 12:37:42
Season: 2008    Day Number: 2    Current Time: 2024-08-23 12:37:42
Season: 2008    Day Number: 3    Current Time: 2024-08-23 12:37:42


...

Season: 2019    Day Number: 61    Current Time: 2024-08-23 14:40:07
Season: 2019    Day Number: 62    Current Time: 2024-08-23 14:40:26
Season: 2019    Day Number: 63    Current Time: 2024-08-23 14:40:29
Season: 2019    Day Number: 64    Current Time: 2024-08-23 14:40:30
Season: 2019    Day Number: 65    Current Time: 2024-08-23 14:40:34
Season: 2019    Day Number: 66    Current Time: 2024-08-23 14:40:41
Season: 2019    Day Number: 67    Current Time: 2024-08-23 14:40:50
Season: 2019    Day Number: 68    Current Time: 2024-08-23 14:40:51
Season: 2019    Day Number: 69    Current Time: 2024-08-23 14:41:11
Season: 2019    Day Number: 70    Current Time: 2024-08-23 14:41:13
Season: 2019    Day Number: 71    Current Time: 2024-08-23 14:41:15
Season: 2019    Day Number: 72    Current Time: 2024-08-23 14:41:19
Season: 2019    Day Number: 73    Current Time: 2024-08-23 14:41:26
Season: 2019    Day Number: 74    Current Time: 2024-08-23 14:41:34
Season: 2019    Day Number: 75    Current Time: 

Season: 2020    Day Number: 48    Current Time: 2024-08-23 14:50:34
Season: 2020    Day Number: 49    Current Time: 2024-08-23 14:50:38
Season: 2020    Day Number: 51    Current Time: 2024-08-23 14:50:40
Season: 2020    Day Number: 53    Current Time: 2024-08-23 14:50:41
Season: 2020    Day Number: 54    Current Time: 2024-08-23 14:50:41
Season: 2020    Day Number: 55    Current Time: 2024-08-23 14:50:47
Season: 2020    Day Number: 56    Current Time: 2024-08-23 14:50:54
Season: 2020    Day Number: 57    Current Time: 2024-08-23 14:50:59
Season: 2020    Day Number: 58    Current Time: 2024-08-23 14:51:01
Season: 2020    Day Number: 59    Current Time: 2024-08-23 14:51:03
Season: 2020    Day Number: 60    Current Time: 2024-08-23 14:51:13
Season: 2020    Day Number: 61    Current Time: 2024-08-23 14:51:15
Season: 2020    Day Number: 62    Current Time: 2024-08-23 14:51:32
Season: 2020    Day Number: 63    Current Time: 2024-08-23 14:51:36
Season: 2020    Day Number: 64    Current Time: 

Season: 2021    Day Number: 64    Current Time: 2024-08-23 15:00:00
Season: 2021    Day Number: 65    Current Time: 2024-08-23 15:00:02
Season: 2021    Day Number: 66    Current Time: 2024-08-23 15:00:05
Season: 2021    Day Number: 67    Current Time: 2024-08-23 15:00:08
Season: 2021    Day Number: 68    Current Time: 2024-08-23 15:00:12
Season: 2021    Day Number: 69    Current Time: 2024-08-23 15:00:24
Season: 2021    Day Number: 70    Current Time: 2024-08-23 15:00:27
Season: 2021    Day Number: 71    Current Time: 2024-08-23 15:00:28
Season: 2021    Day Number: 72    Current Time: 2024-08-23 15:00:30
Season: 2021    Day Number: 73    Current Time: 2024-08-23 15:00:32
Season: 2021    Day Number: 74    Current Time: 2024-08-23 15:00:36
Season: 2021    Day Number: 75    Current Time: 2024-08-23 15:00:41
Season: 2021    Day Number: 76    Current Time: 2024-08-23 15:00:55
Season: 2021    Day Number: 77    Current Time: 2024-08-23 15:00:58
Season: 2021    Day Number: 78    Current Time: 

Season: 2022    Day Number: 60    Current Time: 2024-08-23 15:08:08
Season: 2022    Day Number: 61    Current Time: 2024-08-23 15:08:09
Season: 2022    Day Number: 62    Current Time: 2024-08-23 15:08:15
Season: 2022    Day Number: 63    Current Time: 2024-08-23 15:08:18
Season: 2022    Day Number: 64    Current Time: 2024-08-23 15:08:19
Season: 2022    Day Number: 65    Current Time: 2024-08-23 15:08:23
Season: 2022    Day Number: 66    Current Time: 2024-08-23 15:08:27
Season: 2022    Day Number: 67    Current Time: 2024-08-23 15:08:33
Season: 2022    Day Number: 68    Current Time: 2024-08-23 15:08:35
Season: 2022    Day Number: 69    Current Time: 2024-08-23 15:08:49
Season: 2022    Day Number: 70    Current Time: 2024-08-23 15:08:52
Season: 2022    Day Number: 71    Current Time: 2024-08-23 15:08:54
Season: 2022    Day Number: 72    Current Time: 2024-08-23 15:09:00
Season: 2022    Day Number: 73    Current Time: 2024-08-23 15:09:06
Season: 2022    Day Number: 74    Current Time: 

Season: 2023    Day Number: 53    Current Time: 2024-08-23 15:18:46
Season: 2023    Day Number: 55    Current Time: 2024-08-23 15:18:47
Season: 2023    Day Number: 57    Current Time: 2024-08-23 15:18:48
Season: 2023    Day Number: 58    Current Time: 2024-08-23 15:18:49
Season: 2023    Day Number: 59    Current Time: 2024-08-23 15:18:53
Season: 2023    Day Number: 60    Current Time: 2024-08-23 15:19:03
Season: 2023    Day Number: 61    Current Time: 2024-08-23 15:19:07
Season: 2023    Day Number: 62    Current Time: 2024-08-23 15:19:22
Season: 2023    Day Number: 63    Current Time: 2024-08-23 15:19:24
Season: 2023    Day Number: 64    Current Time: 2024-08-23 15:19:27
Season: 2023    Day Number: 65    Current Time: 2024-08-23 15:19:30
Season: 2023    Day Number: 66    Current Time: 2024-08-23 15:19:37
Season: 2023    Day Number: 67    Current Time: 2024-08-23 15:19:48
Season: 2023    Day Number: 68    Current Time: 2024-08-23 15:19:50
Season: 2023    Day Number: 69    Current Time: 

In [ ]:
# For testing through 2022 only. Comment out for true Project Reset.

# concatenate data from each year 
data = pd.read_csv(rootDirectory + '/Data/Training_Data/Training_Data_' + str(startingSeason + 1) + '.csv')
print(str(int(startingSeason + 1)), data.shape)
for year in range(startingSeason + 2, currentSeason):
    data = pd.concat([data, pd.read_csv(rootDirectory + '/Data/Training_Data/Training_Data_' + str(year) + '.csv')])
    print(str(int(year)), data.shape)
# drop unnecessary columns
data = data.drop(columns=['TeamID A', 'TeamID B'])
# save concatenated data
data.to_csv(rootDirectory + '/Data/Training_Data/All_Training_Data_Thru_2022.csv', index=False)
print('All_Training_Data_Thru_2022.csv written.')

print('Generate training data: completed.')

In [ ]:
# scale training data

# read in training data
# data = pd.read_csv(rootDirectory + '/Data/Training_Data/All_Training_Data.csv')
data = pd.read_csv(rootDirectory + '/Data/Training_Data/All_Training_Data_Thru_2022.csv')

# initialize scaler object
standard_scaler = StandardScaler()

# slice out the 'Season', 'DayNum', TeamAWins', and 'HomeAdv A' columns (these won't be scaled) 
X_unscaled = data.drop(columns=['Season', 'DayNum', 'TeamAWins', 'HomeAdv A'])

# scale the data
X_scaled = standard_scaler.fit_transform(X_unscaled)

# add back the column names
X_scaled = pd.DataFrame(X_scaled, columns=X_unscaled.columns)

# add back the 'TeamAWins' and 'HomeAdv A' columns (which were not scaled)
data_scaled = pd.concat([data.loc[:, ['TeamAWins', 'HomeAdv A']].reset_index(drop=True), X_scaled.reset_index(drop=True)], axis=1)


# # write scaler object (to be used on prediction data)
# with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/standard_scaler.pkl', 'wb') as f:
#     pickle.dump(standard_scaler, f)

# holding out 2023-24 as test set
# write scaler object (to be used on prediction data)
with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/standard_scaler_thru_2022.pkl', 'wb') as f:
    pickle.dump(standard_scaler, f)


In [ ]:
# split data into training / calibration / validation / test set

# split data into 75% training/calibration and 25% validation/test sets
X_train_calib, X_valid_test, y_train_calib, y_valid_test = train_test_split(data_scaled.iloc[:, 1:], 
                                                                            data_scaled['TeamAWins'], 
                                                                            test_size=0.25, 
                                                                            random_state=42)
# split training (60%) / calibration (15%) sets
X_train, X_calib, y_train, y_calib = train_test_split(X_train_calib,
                                                      y_train_calib, 
                                                      test_size=0.2,
                                                      random_state=42)
# split validation (15%) / test (10%) sets
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test,
                                                      y_valid_test, 
                                                      test_size=0.4,
                                                      random_state=42)
print(X_train.shape)
print(X_calib.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_valid.shape)
print(y_test.shape)


In [ ]:
# reduce dimensionality using PCA

# initialize PCA object
pca_object = PCA(n_components=0.95)

# fit PCA object to the training set, transform all 4 sets
X_train_reduced = pca_object.fit_transform(X_train)
X_calib_reduced = pca_object.transform(X_calib)
X_valid_reduced = pca_object.transform(X_valid)
X_test_reduced = pca_object.transform(X_test)


# # write PCA object (to be used on prediction data)
# with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/pca_object.pkl', 'wb') as f:
#     pickle.dump(pca_object, f)

# holding out 2023-24 as test set
# write PCA object (to be used on prediction data)
with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/pca_object_thru_2022.pkl', 'wb') as f:
    pickle.dump(pca_object, f)
    

In [ ]:
# further reduce dimensionality using Recursive Feature Elimination with Cross-Validation

# Define the classifier (using L1 regularization for sparsity)
model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5)

# Perform RFECV
rfecv = RFECV(estimator=model, step=1, cv=cv, scoring='neg_log_loss')
rfecv.fit(X_train_reduced, y_train)

# Get the indices of the selected features
selected_indices = rfecv.support_

# Reduce the feature set based on RFECV
X_train_reduced_selected = X_train_reduced[:, selected_indices]
X_calib_reduced_selected = X_calib_reduced[:, selected_indices]
X_valid_reduced_selected = X_valid_reduced[:, selected_indices]
X_test_reduced_selected = X_test_reduced[:, selected_indices]

# # Print the optimal number of features
# print(f"Optimal number of features: {rfecv.n_features_}")
# print(f"Selected feature indices: {selected_indices}")


# # save selected_indices
# with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/RFECV_selected_indices_thru_2022.pkl', 'wb') as f:
#     pickle.dump(selected_indices, f)

# holding out 2023-24 as test set
# save selected_indices
with open(rootDirectory + '/Data/Scaling_and_Reduction_Objects/RFECV_selected_indices_thru_2022.pkl', 'wb') as f:
    pickle.dump(selected_indices, f)


In [ ]:
# define functions for training / visualization


def compute_calibration_error(y_true, y_pred_proba, n_bins=20):
    """
    Compute Expected Calibration Error (ECE) and Maximum Calibration Error (MCE)
    
    Parameters:
    y_true (array-like): True binary labels.
    y_pred_proba (array-like): Predicted probabilities.
    n_bins (int): Number of bins to use for calibration.
    
    Returns:
    ece (float): Expected Calibration Error.
    mce (float): Maximum Calibration Error.
    """
    # Bins
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(y_pred_proba, bin_edges, right=True)

    # Initialize error metrics
    ece = 0.0
    mce = 0.0

    # Iterate over bins
    for i in range(1, n_bins + 1):
        bin_mask = (bin_indices == i)
        bin_size = np.sum(bin_mask)
        
        if bin_size == 0:
            continue
        
        bin_accuracy = np.mean(y_true[bin_mask])
        bin_confidence = np.mean(y_pred_proba[bin_mask])
        
        # Calculate bin contribution to ECE
        bin_error = np.abs(bin_confidence - bin_accuracy)
        ece += (bin_size / len(y_true)) * bin_error
        mce = max(mce, bin_error)

    return ece, mce




# Custom scoring functions for RandomizedSearchCV
def ece_scorer(estimator, X, y):
    probas = estimator.predict_proba(X)[:, 1]
    ece, _ = compute_calibration_error(y, probas)
    return -ece  # Negative because RandomizedSearchCV maximizes the score

def mce_scorer(estimator, X, y):
    probas = estimator.predict_proba(X)[:, 1]
    _, mce = compute_calibration_error(y, probas)
    return -mce  # Negative because RandomizedSearchCV maximizes the score

# Convert the custom scoring functions to a scikit-learn compatible scorer
ece_scorer_func = make_scorer(ece_scorer, greater_is_better=False)
mce_scorer_func = make_scorer(mce_scorer, greater_is_better=False)




def plot_calib_curves(model, X_test, y_test):

    # make predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Create a figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # plot calibration curve with 'uniform' strategy on the first subplot
    disp1 = CalibrationDisplay.from_predictions(y_valid, y_pred_proba, n_bins=20, strategy='uniform', ax=ax1, label='Uniform')
    ax1.set_title('Calibration Plot (Uniform)')
    ax1.set_xlabel('Mean Predicted Probability')
    ax1.set_ylabel('Fraction of Positives')
    ax1.grid(True)

    # plot calibration curve with 'quantile' strategy on the second subplot
    disp2 = CalibrationDisplay.from_predictions(y_valid, y_pred_proba, n_bins=20, strategy='quantile', ax=ax2, label='Quantile')
    ax2.set_title('Calibration Plot (Quantile)')
    ax2.set_xlabel('Mean Predicted Probability')
    ax2.set_ylabel('Fraction of Positives')
    ax2.grid(True)

    # add a main title for the figure
    plt.suptitle('Calibration Plots')

    # Display the plot
    plt.tight_layout()
    plt.show()
    
    
    

def plot_calib_curves(model, X_test, y_test):
    # make predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Ensure y_test is used consistently with y_pred_proba
    assert len(y_test) == len(y_pred_proba), "Mismatch in number of samples between y_test and y_pred_proba"

    # Create a figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # plot calibration curve with 'uniform' strategy on the first subplot
    disp1 = CalibrationDisplay.from_predictions(y_test, y_pred_proba, n_bins=20, strategy='uniform', ax=ax1, label='Uniform')
    ax1.set_title('Calibration Plot (Uniform)')
    ax1.set_xlabel('Mean Predicted Probability')
    ax1.set_ylabel('Fraction of Positives')
    ax1.grid(True)

    # plot calibration curve with 'quantile' strategy on the second subplot
    disp2 = CalibrationDisplay.from_predictions(y_test, y_pred_proba, n_bins=20, strategy='quantile', ax=ax2, label='Quantile')
    ax2.set_title('Calibration Plot (Quantile)')
    ax2.set_xlabel('Mean Predicted Probability')
    ax2.set_ylabel('Fraction of Positives')
    ax2.grid(True)

    # add a main title for the figure
    plt.suptitle('Calibration Plots')

    # Display the plot
    plt.tight_layout()
    plt.show()

    
    

In [ ]:
# train logistic regression model

# create and fit model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_reduced, y_train)

# make predictions
y_pred_proba = log_reg_model.predict_proba(X_valid_reduced)[:, 1]

# measure performance
log_reg_log_loss_score = log_loss(y_valid, y_pred_proba)
print(f'Log Loss: {log_reg_log_loss_score}')
log_reg_brier_score = brier_score_loss(y_valid, y_pred_proba)
print(f'Brier Score: {log_reg_brier_score}')
log_reg_roc_auc = roc_auc_score(y_valid, y_pred_proba)
print(f'ROC AUC Score: {log_reg_roc_auc}')
ece, mce = compute_calibration_error(y_valid, y_pred_proba, n_bins=20)
print(f'Expected Calibration Error (ECE): {ece}')
print(f'Maximum Calibration Error (MCE): {mce}')

# plot calibration curves
plot_calib_curves(log_reg_model, X_valid_reduced, y_valid)


# save model
# with open(rootDirectory + '/Models/log_reg_model.pkl', 'wb') as f:
#     pickle.dump(log_reg_model, f)

# holding out 2023-24 as test set
# save model
with open(rootDirectory + '/Models/log_reg_model_thru_2022.pkl', 'wb') as f:
    pickle.dump(log_reg_model, f)


In [ ]:
# random search for best random forest parameters

# Define the parameter space for Random Forest
param_dist = {
    'n_estimators': randint(10, 2000),
    'max_depth': randint(1, 25),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'bootstrap': [True, False]
}

# Number of iterations
n_iter_search = 50

# Track best model and scores
best_ece = float('inf')
best_mce = float('inf')
best_params = None
best_ece_rf = None
best_mce_rf = None

# Perform Randomized Search
random_sampler = ParameterSampler(param_dist, n_iter=n_iter_search, random_state=42)

# Verbose logging
print(f"Starting Randomized Search with {n_iter_search} iterations...\n")
start_time = time.time()

for idx, params in enumerate(random_sampler):
    iteration_start_time = time.time()
    print(f"Iteration {idx+1}/{n_iter_search}")
    print(f"Current Parameters: {params}")

    # Train the model with current parameters
    rf = RandomForestClassifier(**params, random_state=42)
    rf.fit(X_train_reduced, y_train)

    # Calibrate the model with calibration set
    calibrated_rf = CalibratedClassifierCV(estimator=rf, method='sigmoid', cv='prefit')
    calibrated_rf.fit(X_calib_reduced, y_calib)
    
    # Evaluate on validation set
    y_pred_proba = calibrated_rf.predict_proba(X_valid_reduced)[:, 1]
    ece, mce = compute_calibration_error(y_valid, y_pred_proba, n_bins=20)

    # Verbose logging for evaluation results
    print(f"ECE: {ece}, MCE: {mce}")
    iteration_duration = time.time() - iteration_start_time
    print(f"Iteration {idx+1} completed in {iteration_duration:.2f} seconds\n")

    # Update best model based on ECE and MCE
    if ece < best_ece:
        best_ece = ece
        cooresp_mce = mce
        best_ece_params = params
        best_ece_rf = calibrated_rf
    if mce < best_mce:
        best_mce = mce
        cooresp_ece = ece
        best_mce_params = params
        best_mce_rf = calibrated_rf

        
# Output the best scores and parameters
total_duration = time.time() - start_time
print(f"Randomized Search completed in {total_duration:.2f} seconds")
print(f'Best ECE Model: ECE={best_ece}, MCE={cooresp_mce}')
print(f'Best ECE parameters: {best_ece_params}')
print(f'Best MCE Model: ECE={cooresp_ece}, MCE={best_mce}')
print(f'Best MCE parameters: {best_mce_params}')


# save best models
# with open(rootDirectory + '/Models/best_ece_rf_rand_search.pkl', 'wb') as f:
#     pickle.dump(best_ece_rf, f)
# with open(rootDirectory + '/Models/best_mce_rf_rand_search.pkl', 'wb') as f:
#     pickle.dump(best_mce_rf, f)

# holding out 2023-24 as test set
# save best models
with open(rootDirectory + '/Models/best_ece_rf_rand_search_thru_2022.pkl', 'wb') as f:
    pickle.dump(best_ece_rf, f)
with open(rootDirectory + '/Models/best_mce_rf_rand_search_thru_2022.pkl', 'wb') as f:
    pickle.dump(best_mce_rf, f)


In [ ]:
# refined grid search for best ECE random forest

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [250, 260, 270, 280],  # Centered around 262
    'max_depth': [20, 21, 22, 23],         # Centered around 21
    'min_samples_split': [5, 6, 7, 8],     # Centered around 7
    'min_samples_leaf': [1, 2, 3],         # Centered around 1
    'bootstrap': [True, False]             # Include both options for variety
}

# Track best model and scores
best_ece = float('inf')
best_mce = float('inf')
best_params_ece = None
best_params_mce = None
best_ece_rf = None
best_mce_rf = None

# Perform Grid Search
param_list = list(ParameterGrid(param_grid))
n_iter_search = len(param_list)
print(f"Starting Grid Search with {n_iter_search} combinations...\n")
start_time = time.time()

for idx, params in enumerate(param_list):
    iteration_start_time = time.time()
    print(f"Iteration {idx+1}/{n_iter_search}")
    print(f"Current Parameters: {params}")

    # Train the model with current parameters
    rf = RandomForestClassifier(**params, random_state=42)
    rf.fit(X_train_reduced, y_train)

    # Calibrate the model with calibration set
    calibrated_rf = CalibratedClassifierCV(estimator=rf, method='sigmoid', cv='prefit')
    calibrated_rf.fit(X_calib_reduced, y_calib)
    
    # Evaluate on validation set
    y_pred_proba = calibrated_rf.predict_proba(X_valid_reduced)[:, 1]
    ece, mce = compute_calibration_error(y_valid, y_pred_proba, n_bins=20)

    # Verbose logging for evaluation results
    print(f"ECE: {ece}, MCE: {mce}")
    iteration_duration = time.time() - iteration_start_time
    print(f"Iteration {idx+1} completed in {iteration_duration:.2f} seconds\n")

    # Update best model based on ECE and MCE
    if ece < best_ece:
        best_ece = ece
        cooresp_mce = mce
        best_params_ece = params
        best_ece_rf = calibrated_rf
    if mce < best_mce:
        best_mce = mce
        cooresp_ece = ece
        best_params_mce = params
        best_mce_rf = calibrated_rf

        
# Output the best scores and parameters
total_duration = time.time() - start_time
print(f"Grid Search completed in {total_duration:.2f} seconds")
print(f'Best ECE model: ECE={best_ece}, MCE={cooresp_mce}')
print(f'Best ECE parameters: {best_params_ece}')
print(f'Best MCE model: ECE={cooresp_ece}, MCE={best_mce}')
print(f'Best MCE parameters: {best_params_mce}')


# save best models
import pickle
with open(rootDirectory + '/Models/best_ece_rf_grid_search_ece.pkl', 'wb') as f:
    pickle.dump(best_ece_rf, f)
with open(rootDirectory + '/Models/best_mce_rf_grid_search_ece.pkl', 'wb') as f:
    pickle.dump(best_mce_rf, f)
    

In [ ]:
# train specific random forest

params = {
    'n_estimators': 262,
    'max_depth': 21,
    'min_samples_split': 7,
    'min_samples_leaf': 1,
    'bootstrap': False
}

# Train the model with current parameters
rf = RandomForestClassifier(**params, random_state=42)
rf.fit(X_train_reduced_selected, y_train)

# Calibrate the model with calibration set
calibrated_rf = CalibratedClassifierCV(estimator=rf, method='sigmoid', cv='prefit')
calibrated_rf.fit(X_calib_reduced_selected, y_calib)

# Evaluate on validation set
y_pred_proba = calibrated_rf.predict_proba(X_valid_reduced_selected)[:, 1]
ece, mce = compute_calibration_error(y_valid, y_pred_proba, n_bins=20)

print('ece', ece, 'mce', mce)

In [ ]:
# random search for best neural network parameters

def create_model(optimizer='adam', activation='relu', num_layers=1, neurons_range=(166, 512), dropout_rate=0.0, learning_rate=0.001):
    # Start with an Input layer
    input_layer = Input(shape=(X_train_reduced.shape[1],))
    model = Sequential()
    model.add(input_layer)
    
    # Add the first Dense layer
    neurons = randint(neurons_range[0], neurons_range[1]).rvs()
    model.add(Dense(neurons, activation=activation))
    model.add(Dropout(dropout_rate))
    
    # Add additional hidden layers
    for _ in range(num_layers - 1):
        neurons = randint(neurons_range[0], neurons_range[1]).rvs()
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(dropout_rate))
    
    # Add the output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Configure the optimizer with the specified learning rate
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    # Add other optimizers as needed
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Custom callback for early stopping based on ECE and MCE
class CalibrationEarlyStopping(Callback):
    def __init__(self, X_calib, y_calib, X_valid, y_valid, patience=10):
        super().__init__()
        self.X_calib = X_calib
        self.y_calib = y_calib
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.patience = patience
        self.best_ece = float('inf')
        self.best_mce = float('inf')
        self.wait = 0
        self.best_weights_ece = None
        self.best_weights_mce = None
        self.wait_ece = 0
        self.wait_mce = 0

    def on_epoch_end(self, epoch, logs=None):
        # Calibrate the model with the calibration set
        calibrated_model = CalibratedClassifierCV(base_estimator=self.model, method='sigmoid', cv='prefit')
        calibrated_model.fit(self.X_calib, self.y_calib)
        
        # Evaluate on the validation set
        y_pred_proba = calibrated_model.predict_proba(self.X_valid)[:, 1]
        ece, mce = compute_calibration_error(self.y_valid, y_pred_proba)

        # Check if we need to update the best model based on ECE
        improved = False
        if ece < self.best_ece:
            self.best_ece = ece
            self.best_weights_ece = self.model.get_weights()
            self.wait_ece = 0  # Reset patience counter for ECE
            improved = True
        else:
            self.wait_ece += 1

        # Check if we need to update the best model based on MCE
        if mce < self.best_mce:
            self.best_mce = mce
            self.best_weights_mce = self.model.get_weights()
            self.wait_mce = 0  # Reset patience counter for MCE
            improved = True
        else:
            self.wait_mce += 1

        # Early stopping check: stop if both ECE and MCE do not improve
        if not improved and self.wait_ece >= self.patience and self.wait_mce >= self.patience:
            self.model.stop_training = True

        print(f"[Epoch {epoch + 1}] ECE: {ece}, MCE: {mce}, Best ECE: {self.best_ece}, Best MCE: {self.best_mce}")

# Define the parameter grid
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['relu', 'tanh'],
    'num_layers': randint(2, 10),
    'neurons_range': [(166, 512)],
    'dropout_rate': uniform(0, 0.5),
    'batch_size': [32, 64, 128],
    'epochs': [100, 150, 200],
    'learning_rate': uniform(0.001, 0.1)
}

# Initialize variables to store the best scores and parameters
best_ece = float('inf')
best_mce = float('inf')
best_params_ece = None
best_params_mce = None
best_ece_ann = None
best_mce_ann = None

# Perform Randomized Search
random_sampler = ParameterSampler(param_grid, n_iter=50, random_state=42)
n_iter_search = len(random_sampler)
start_time = time.time()
print(f"Starting Randomized Search with {n_iter_search} iterations...\n")

for idx, params in enumerate(random_sampler):
    iteration_start_time = time.time()
    print(f"\n===== Iteration {idx + 1}/{n_iter_search} =====")
    print(f"Current Parameters: {params}")

    # Create a Keras model with the sampled parameters
    model = KerasClassifier(build_fn=create_model, **params, verbose=0)
    
    # Set up early stopping based on calibration metrics
    calibration_early_stopping = CalibrationEarlyStopping(X_calib_reduced, y_calib, X_valid_reduced, y_valid, patience=10)

    # Train the model on the training set
    print(f"Training model...")
    train_start_time = time.time()
    history = model.fit(X_train_reduced, y_train, epochs=params['epochs'], batch_size=params['batch_size'],
                        validation_data=(X_valid_reduced, y_valid), callbacks=[calibration_early_stopping], verbose=1)
    train_duration = time.time() - train_start_time
    print(f"Training completed in {train_duration:.2f} seconds")

    # Save the best models based on ECE and MCE
    if calibration_early_stopping.best_ece < best_ece:
        best_ece = calibration_early_stopping.best_ece
        best_params_ece = params
        best_ece_ann = model.model  # Save the model with the best ECE

    if calibration_early_stopping.best_mce < best_mce:
        best_mce = calibration_early_stopping.best_mce
        best_params_mce = params
        best_mce_ann = model.model  # Save the model with the best MCE

    print(f"Best ECE for this iteration: {calibration_early_stopping.best_ece}")
    print(f"Best MCE for this iteration: {calibration_early_stopping.best_mce}")
    iteration_duration = time.time() - iteration_start_time
    print(f"Iteration {idx + 1} completed in {iteration_duration:.2f} seconds\n")

    
# Save the best models based on ECE and MCE
print("Saving best models...")
with open(rootDirectory + '/Models/best_ece_nn_rand_search.pkl', 'wb') as f:
    pickle.dump(best_ece_ann, f)
with open(rootDirectory + '/Models/best_mce_nn_rand_search.pkl', 'wb') as f:
    pickle.dump(best_mce_ann, f)

# Output the best scores and parameters
total_duration = time.time() - start_time
print(f"Randomized Search completed in {total_duration:.2f} seconds")
print(f'Best ECE model: ECE={compute_calibration_error(y_valid, best_ece_ann.predict_proba(X_valid_reduced)[:, 1])[0]}, MCE={compute_calibration_error(y_valid, best_ece_ann.predict_proba(X_valid_reduced)[:, 1])[1]}, with parameters: {best_params_ece}')
print(f'Best MCE model: ECE={compute_calibration_error(y_valid, best_mce_ann.predict_proba(X_valid_reduced)[:, 1])[0]}, MCE={compute_calibration_error(y_valid, best_mce_ann.predict_proba(X_valid_reduced)[:, 1])[1]}, with parameters: {best_params_mce}')


In [ ]:
# random search for best XGBoost parameters

def should_stop_early(validation_scores, patience):
    if len(validation_scores) < patience:
        return False
    recent_scores = validation_scores[-patience:]
    return all(recent_scores[i] >= recent_scores[i-1] for i in range(1, patience))

def perform_random_search_with_cross_validation(X_train, y_train, X_calib, y_calib, X_valid, y_valid, n_iter_search=50, patience=5, n_splits=5):
    # Define the parameter space for XGBoost
    param_dist = {
        'n_estimators': randint(50, 500),  # Reasonable range for n_estimators
        'max_depth': randint(3, 8),  # Narrower range for max_depth
        'learning_rate': uniform(0.05, 0.15),  # Now in [0.05, 0.20]
        'subsample': uniform(0.7, 0.3),  # Now in [0.7, 1.0]
        'colsample_bytree': uniform(0.7, 0.3),  # Now in [0.7, 1.0]
        'colsample_bylevel': uniform(0.7, 0.3),  # Now in [0.7, 1.0]
        'colsample_bynode': uniform(0.7, 0.3),  # Now in [0.7, 1.0]
        'min_child_weight': randint(1, 6),  # Reasonable range for min_child_weight
        'reg_alpha': uniform(0.0, 0.5),  # L1 regularization term, reduced range
        'reg_lambda': uniform(0.0, 0.5),  # L2 regularization term, reduced range
    }

    best_ece = float('inf')
    best_mce = float('inf')
    best_ece_model = None
    best_mce_model = None
    validation_scores = []

    random_sampler = ParameterSampler(param_dist, n_iter=n_iter_search, random_state=42)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    print(f"Starting Randomized Search with {n_iter_search} iterations and {n_splits}-fold cross-validation...\n")
    start_time = time.time()

    for idx, params in enumerate(random_sampler):
        iteration_start_time = time.time()
        print(f"Iteration {idx+1}/{n_iter_search}")
        print(f"Current Parameters: {params}")

        xgb_model = xgb.XGBClassifier(**params, random_state=42, eval_metric='logloss')

        # Perform cross-validation
        cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=skf, scoring='neg_log_loss')
        avg_cv_score = np.mean(cv_scores)

        xgb_model.fit(X_train, y_train)

        # Calibrate the model with the calibration set
        calibrated_xgb = CalibratedClassifierCV(estimator=xgb_model, method='sigmoid', cv='prefit')
        calibrated_xgb.fit(X_calib, y_calib)

        # Evaluate on validation set
        y_pred_proba = calibrated_xgb.predict_proba(X_valid)[:, 1]
        ece, mce = compute_calibration_error(y_valid, y_pred_proba, n_bins=20)

        print(f"ECE: {ece}, MCE: {mce}, CV Score: {avg_cv_score}")
        iteration_duration = time.time() - iteration_start_time
        print(f"Iteration {idx+1} completed in {iteration_duration:.2f} seconds\n")

        validation_scores.append(ece)

        if ece < best_ece:
            best_ece = ece
            best_ece_model = calibrated_xgb
        if mce < best_mce:
            best_mce = mce
            best_mce_model = calibrated_xgb

        if should_stop_early(validation_scores, patience):
            print("Early stopping triggered.")
            break

    total_duration = time.time() - start_time
    print(f"Randomized Search completed in {total_duration:.2f} seconds")
    print(f'Best ECE Model: ECE={best_ece}, MCE={mce}')
    print(f'Best MCE Model: ECE={ece}, MCE={best_mce}')

    
    # Save the best models
    with open(rootDirectory + '/Models/best_ece_xgb_rand_search.pkl', 'wb') as f:
        pickle.dump(best_ece_model, f)
    with open(rootDirectory + '/Models/best_mce_xgb_rand_search.pkl', 'wb') as f:
        pickle.dump(best_mce_model, f)

    return best_ece_model, best_mce_model



# Use the function to perform random search with cross-validation and early stopping
best_ece_model, best_mce_model = perform_random_search_with_cross_validation(
    X_train_reduced_selected, y_train,
    X_calib_reduced_selected, y_calib,
    X_valid_reduced_selected, y_valid
)


# evaluate on the test set using the selected models
y_pred_proba_test_ece = best_ece_model.predict_proba(X_test_reduced_selected)[:, 1]
ece_final, mce_final_ece = compute_calibration_error(y_test, y_pred_proba_test_ece, n_bins=20)
print(f"Final ECE on Test Set: {ece_final}, Final MCE: {mce_final_ece}")

y_pred_proba_test_mce = best_mce_model.predict_proba(X_test_reduced_selected)[:, 1]
ece_final_mce, mce_final = compute_calibration_error(y_test, y_pred_proba_test_mce, n_bins=20)
print(f"Final ECE on Test Set: {ece_final_mce}, Final MCE: {mce_final}")


In [ ]:
# Evaluate final model on test set

with open(rootDirectory + '/Models/final_model.pkl', 'rb') as file:
    model = pickle.load(file)

# make predictions
y_pred_proba = model.predict_proba(X_test_reduced_selected)[:, 1]

# print calibration error metrics
print(compute_calibration_error(y_test, y_pred_proba, n_bins=20))

# plot calibration curve
plot_calib_curves(model, X_test_reduced_selected, y_test)
